# Data Visualisation

## Notes and Quick ideas

- Now do plots with the repetitions baked in. Show which hyperparameter combinations have the least variance

- Figure out what went wrong in the labelling
    - Potentially relabel validation/training/testing(!)
- Redo difference graph after the relabelling
- Give a description of the dataset, what's in it
- Load dataset onto zenodo
- Create a bridge from background chapter to how the models are used

### Meeting notes
- [x] Check (before writing results chapter) that the delay isn't too big
- [x] Make *very* sure that the model can be run in real time, with the gloves
- [x] Conf matrix should be %age of the class
- [x] Explain Conf matrix structure (diagonals/orientations/fingers) in thesis
- [x] Include 'dummy' models which perfectly predict only finger/orientation/hand, etc
     - put it in a separate section in methodology
- [x] Look into plotting error on FFNN
- Discuss precision/recall for 51 gesture FFNN/HMM/CuSUM  -> Why would this happen
- Error types: (wrong timestep) x (wrong gesture)
    - It seems like the FFNN is not getting the timestep wrong, it's just wrong
- Explore plots of hpars affecting regularization and validation performance
- Make note that the HMM is only predicting 200 g255 gestures

### Changes made

- F1 score was being set to NaN, resulting in the average being too high (and F1 ~= 1.0)
- Grid search was unable to explore the search space fully, so [Optuna](https://optuna.readthedocs.io/en/stable/) was used for the search.
    - Specifically, the [Tree-structured Parzen Estimator](https://optuna.readthedocs.io/en/stable/reference/samplers/generated/optuna.samplers.TPESampler.html#optuna.samplers.TPESampler) performs the search. Bergstra, James et al. “Algorithms for Hyper-Parameter Optimization.” NIPS (2011). [Explanatory Blog](http://neupy.com/2016/12/17/hyperparameter_optimization_for_neural_networks.html#tree-structured-parzen-estimators-tpe)
- 


# Data Preparation

In [ ]:
# Change directory to keep paths consistent
%cd /Users/brk/projects/masters/SU/ergo/src

## Imports and setup

In [ ]:
# minimise me
%load_ext autoreload
%autoreload 2
import seaborn as sns
import seaborn.objects as so
import matplotlib.pyplot as plt
import matplotlib as mpl
import ipywidgets as widgets
import datetime
from ipywidgets import interact, interactive, fixed, interact_manual
import pandas as pd
import numpy as np
import models
import vis
import common
import read
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
import sklearn
import tqdm
import logging as l
import tqdm
import yaml
import glob
from matplotlib.colors import LogNorm
import re
from sklearn.metrics import classification_report
from scipy.stats import f

mpl.rc('font', family='serif', serif='cmr10')
mpl.rc('axes.formatter', use_mathtext=True)


## Utility functions

In [ ]:
# minimise me
def prettify_col_name(x):
    return x.split('.')[-1].replace('_', ' ').title()

def calculate_prediction_ellipse(x, y, alpha=0.95):
    """Given some x and y data, calculate the (1-alpha) confidence ellipse."""
    data = np.column_stack((x, y)) # Combine x and y into a single data array
    num_dimensions = data.shape[1]
    num_data_points = data.shape[0]
    # Estimate the sample covariance matrix
    sample_covariance_matrix = np.cov(data, rowvar=False)
    # Calculate the sample mean for each dimension
    sample_mean = np.mean(data, axis=0)
    # Generate angles for the ellipse
    theta = np.linspace(0, 2*np.pi, num=100)
    # Calculate the radius of the ellipse. `f.ppf` is the inverse of the CDF
    radius = np.sqrt(
        num_dimensions * (num_data_points - 1) / (num_data_points - num_dimensions) *
        (1 + 1/num_data_points) * f.ppf(1 - alpha, num_dimensions, num_data_points - num_dimensions)
    )
#     print(sample_covariance_matrix)
    # Compute the Cholesky decomposition of the covariance matrix
    chol_cov_matrix = np.linalg.cholesky(sample_covariance_matrix)
    # Generate ellipse offset based on Cholesky decomposition
    ellipse_offset = np.outer(np.cos(theta), chol_cov_matrix[0, :]) + np.outer(np.sin(theta), chol_cov_matrix[1, :])
    # Calculate the points of the prediction interval ellipse
    prediction_ellipse_points = sample_mean + radius * ellipse_offset
    return prediction_ellipse_points

def get_npz_data_from_model(model_dir):
    """Given a directory of a model, return it's y_pred and y_true."""
    data = np.load(f'{model_dir}/y_val_true_y_val_pred.npz')
    y_true = data['y_true']
    y_pred = data['y_pred']
    return y_true, y_pred

def show_conf_mat_from_model(model_dir, ax=None):
    """Given a directory of a model, plot its confidence matrix"""
    y_true, y_pred = get_npz_data_from_model(model_dir)
    cm_val = tf.math.confusion_matrix(
        y_true.flatten(), 
        y_pred.flatten()
    ).numpy()
    p = vis.conf_mat(cm_val / cm_val.sum(axis=0), ax=ax)
    return p

## Load data

In [ ]:
# Read in data from hpar optimisation
paths = sorted(glob.glob('../saved_models/results_*_optuna.jsonl'))
print(f'Reading data from\n', "\n".join(paths))
dfs = map(
    lambda path: pd.read_json(path, lines=True),
    paths
)
# Concat the dataframes together, and then do a 
# copy to avoid a dataframe fragmentation warning
# Reset the index to avoid a seaborn error https://github.com/mwaskom/seaborn/issues/3291
df = pd.concat(dfs).reset_index(drop=True).copy()
df['preprocessing.num_gesture_classes'] = df['preprocessing.num_gesture_classes'].fillna('51')
df['preprocessing.num_gesture_classes'] = df['preprocessing.num_gesture_classes'].astype(int).astype(str)

# 50-class HFFNNs don't make sense, remove them
df = df[~(
    (df['model_type'] == 'HFFNN')
    & (df['preprocessing.num_gesture_classes'] == '50')
)]

df.groupby(['model_type', 'preprocessing.num_gesture_classes']).size()

## Constants to keep colours consistent

In [ ]:
model_colours = {
    'FFNN': 'tab:blue',
    'HFFNN': 'tab:orange',
    'CuSUM': 'tab:green',
    'HMM': 'tab:red',
    'SVM': 'tab:purple',
}
palette = 'Spectral'
other_colours = [
    'tab:brown',
    'tab:pink',
    'tab:grey',
    'tab:olive',
    'tab:cyan',
]

## Calculate some auxillary values

In [ ]:
# Preprocess the data a little bit, and get a list of dependant variables
# Preprocess the df a bit to get some nice-to-use columns

prefixes = (
    'ffnn.nodes_per_layer',
    'hffnn.majority.ffnn.nodes_per_layer',
    'hffnn.minority.ffnn.nodes_per_layer',
)
for i in (1, 2, 3):
    for prefix in prefixes:
        print(f'{prefix}.{i}')
        df[f'{prefix}.{i}'] = df[prefix].apply(
            lambda x: x[i-1] if isinstance(x, list) and len(x) >= i else None
        )

# Calculate ratios
avgs = ('macro avg', 'weighted avg')
metrics = ('f1-score', 'precision', 'recall')

for avg in avgs:
    for metric in metrics:
        df[f'ratio.{avg}.{metric}'] = df[f'trn.{avg}.{metric}'] / df[f'val.{avg}.{metric}']
        df[f'ratio.{avg}.{metric}'] = np.where(
            np.isfinite(df[f'ratio.{avg}.{metric}']),
            df[f'ratio.{avg}.{metric}'],
            np.nan
        )

# Print out a list of dependant variables
dep_vars = sorted([
    c for c in df.columns 
    if 'val' not in c and 'trn' not in c and 'ratio' not in c and c not in (
        'saved_at', 'fit_time', 'preprocessing.gesture_allowlist', 
)], key=lambda c: str(c))
print(f"Dependant variables: {dep_vars}")
# print("\nVariables which change:")
# max_len = max(map(lambda x: len(x), dep_vars))
# Print out all dependant variables that change
# for var in dep_vars:
#     uniq = df[var].apply(lambda x: str(x) if isinstance(x, list) else x).unique()
#     if len(uniq) > 1:
#         print(f"{var: <{max_len}} {uniq}")
        
df['ffnn.dropout_rate'] = np.round(df['ffnn.dropout_rate'], 6)

df['val.pred_time_per_obs'] = df['val.pred_time'] / df['val.num_observations']
df['trn.pred_time_per_obs'] = df['trn.pred_time'] / df['trn.num_observations']
df['fit_time_per_obs'] = df['fit_time'] / df['trn.num_observations']


# Add some log10 columns
log10_cols = [
    'val.loss',
    'trn.loss',
    'ffnn.l2_coefficient',
    'nn.batch_size',
    'nn.learning_rate',
]
df[[f'{c}.log10' for c in log10_cols]] = np.log10(df[log10_cols])

# There are a *lot* of columns. Here's a more-useful subset
subset_cols = [
    c for c in df.columns
    if (not re.search(r'((trn|val)\.\d+\.)|weighted avg', c)) and 
        (c not in [
            'hmm', 'lstm', 'ffnn', 'nn', 'hffnn', 'cusum', 'svm',
            'preprocessing.n_timesteps',
            'preprocessing.gesture_allowlist',
            'preprocessing.gesture_allowlist',
        ])
]




In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(8, 8))
ngestures = ('51', '50', '5')
xmin = None
xmax = None
for ax, ngesture in zip(axs, ngestures):
    sns.scatterplot(
        data=df[df['preprocessing.num_gesture_classes'] == ngesture],
        x='saved_at',
        y='preprocessing.seed',
        hue='model_type',
        s=10,
        hue_order=list(model_colours.keys()),
    #     alpha=0.1,
        ax=ax,
    )
    ax.set_title(f'{ngesture} gestures')
#     if xmin is None: xmin = ax.get_xlim()[0]
#     if xmax is None: xmax = ax.get_xlim()[1]
#     xmin = min(xmin, ax.get_xlim()[0])
#     xmax = min(xmax, ax.get_xlim()[1])
# for ax in axs:
#     ax.set_xlim((xmin, xmax))
plt.tight_layout()

In [ ]:
print(df.shape)
df = df[
    (df['model_type'] != 'HFFNN')
    | (df['saved_at'] > pd.to_datetime('2023-10-01T11:00:00'))
]
print(df.shape)


# Plotting

## Bar plot of number of observations

In [ ]:
(
    X_trn, X_val, y_trn, y_val, dt_trn, dt_val
) = common.read_and_split_from_npz("../gesture_data/trn_20_10.npz")

fig, axs = plt.subplots(1, 2, figsize=(8, 4))
counts = pd.Series(y_trn).value_counts()
axs[0].bar(counts.index, counts)


counts = pd.Series(y_trn[y_trn != 50]).value_counts()
axs[1].bar(counts.index, counts)

axs[0].set(
    title='Count of classes in the training dataset\n',
    xlabel='Class',
    ylabel='Count',
)
axs[1].set(
    title='Count of classes in the training dataset\n(excluding class 50)',
    xlabel='Class',
    ylabel='Count',
)
plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_class_imbalance.pdf'
)

## Precision vs Recall vs $F_1$

### 51 classes, Precision vs Recall vs $F_1$

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(10, 5))

recall_grid, precision_grid = np.meshgrid(
    np.linspace(0, 1, 100), 
    np.linspace(0, 1, 100)
)
f1_score = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)

contours = axs[0].contour(
    recall_grid, 
    precision_grid,
    f1_score, 
    levels=np.linspace(0.1, 1, 10), 
    colors='black',
    alpha=0.25
)
axs[0].clabel(contours, inline=True, fontsize=8, fmt='%.2f')

sns.scatterplot(
    data=df[df['preprocessing.num_gesture_classes'] == '51'],
    x='val.macro avg.precision',
    y='val.macro avg.recall',
    s=5,
    alpha=0.5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    ax=axs[0],
)
axs[0].set_xlim((-0.1, 1.1))
axs[0].set_ylim((-0.1, 1.1))
axs[0].plot([0,1], [0,1], color='black', alpha=.1)
axs[0].set_title(f'Precision vs Recall for models trained on 51 classes\n(contours denote $F_1$)')
axs[0].set_xlabel(f'Precision')
axs[0].set_ylabel(f'Recall')
axs[0].legend().set_title("Model ")


sns.stripplot(
    data=df[df['preprocessing.num_gesture_classes'] == '51'],
    y='val.macro avg.f1-score',
    x='model_type',
    s=2,
    alpha=0.5,
    order=list(model_colours.keys()),
    hue='model_type',
    hue_order=list(model_colours.keys()),
    ax=axs[1],
    legend=False,
)
axs[1].set_title(f'$F_1$-score for models trained on 51 classes')
axs[1].set_xlabel(f'Model Type')
axs[1].set_ylabel(f'$F_1$-score')
axs[1].set_ylim((-0.1, 1.1))
axs[1].grid(axis='y')

plt.savefig(
    '../../report/src/imgs/graphs/05_precision_recall_51_classes.pdf', 
    bbox_inches='tight'
)
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))

data = df.loc[
    df['preprocessing.num_gesture_classes'] == '51',
    ['model_type', 'val.macro avg.recall', 'val.macro avg.precision']
].melt(
    id_vars=['model_type'], 
    var_name='metric', 
    value_name='value'
)
data['metric'] = data['metric'].replace({
    'val.macro avg.recall': 'Recall',
    'val.macro avg.precision': 'Precision',
})

sns.stripplot(
    data=data,
    x="model_type", 
    y="value", 
    hue="metric",
    order=list(model_colours.keys()),
    palette=palette,
    dodge=True,
    alpha=0.75,
    size=2,
    ax=ax
)
ax.set_ylim((-0.05, 1.05))

ax.set_title(f'Precision and recall for all model types')
ax.set_xlabel(f'Model Type')
ax.set_ylabel(f'Metric Value')
ax.legend().set_title("Metric")
ax.set_yticks(np.arange(0, 1.1, .1))
ax.set_yticklabels(np.round(np.arange(0, 1.1, .1), 1))
ax.grid(axis='y')

plt.savefig(
    '../../report/src/imgs/graphs/05_precision_recall_stripplot.pdf', 
    bbox_inches='tight'
)
plt.tight_layout()

### Precision vs Recall for all models

In [ ]:
# Three plots showing the precision and recall for all models.
# Each plot showing either 51, 50, or 5 gesture classes
fig, axs = plt.subplots(1, 3, figsize=(12, 4))
n_gesture_classes = ('51', '50', '5')

recall_grid, precision_grid = np.meshgrid(
    np.linspace(0, 1, 100), 
    np.linspace(0, 1, 100)
)
f1_score = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)

for ax, n_classes in zip(axs, n_gesture_classes):
    contours = ax.contour(
        recall_grid, 
        precision_grid,
        f1_score, 
        levels=np.linspace(0.1, 1, 10), 
        colors='black',
        alpha=0.25
    )
    ax.clabel(contours, inline=True, fontsize=8, fmt='%.2f')
    
    sns.scatterplot(
        data=df[df['preprocessing.num_gesture_classes'] == n_classes],
        x='val.macro avg.precision',
        y='val.macro avg.recall',
        s=10,
        alpha=0.5,
        hue='model_type',
        hue_order=list(model_colours.keys()),
        ax=ax,
    )
    ax.set_xlim((-0.1, 1.1))
    ax.set_ylim((-0.1, 1.1))
    ax.set_xlabel('Precision')
    ax.set_ylabel('Recall')
    ax.plot([0,1], [0,1], color='black', alpha=.1)
    ax.set_title(f'{n_classes} gesture classes')
    ax.legend().set_title('Model Type')

plt.tight_layout()

## Best model by highest lower 90th percentile

In [ ]:
from scipy import stats
type_to_hpars = {
    'CuSUM': ['cusum.thresh'],
    'HMM': ['hmm.covariance_type'],
    'FFNN': [
        'ffnn.dropout_rate',
        'ffnn.l2_coefficient',
        'ffnn.nodes_per_layer.1',
        'ffnn.nodes_per_layer.2',
        'ffnn.nodes_per_layer.3',
        'nn.batch_size',
        'nn.learning_rate',
    ],
    'HFFNN': [
        'hffnn.majority.ffnn.dropout_rate',
        'hffnn.majority.ffnn.l2_coefficient',
        'hffnn.majority.ffnn.nodes_per_layer.1',
        'hffnn.majority.ffnn.nodes_per_layer.2',
        'hffnn.majority.ffnn.nodes_per_layer.3',
        'hffnn.majority.nn.epochs',
        'hffnn.majority.nn.batch_size',
        'hffnn.majority.nn.learning_rate',
        'hffnn.minority.ffnn.dropout_rate',
        'hffnn.minority.ffnn.l2_coefficient',
        'hffnn.minority.ffnn.nodes_per_layer.1',
        'hffnn.minority.ffnn.nodes_per_layer.2',
        'hffnn.minority.ffnn.nodes_per_layer.3',
        'hffnn.minority.nn.epochs',
        'hffnn.minority.nn.batch_size',
        'hffnn.minority.nn.learning_rate',
    ],
    'SVM': ['svm.c', 'svm.class_weight'],
}

def tenth_conf_interval(series):
    mean = np.mean(series)
    sem = stats.sem(series)
    if sem == 0:
        return mean
    confidence_interval = stats.t.interval(
        0.90, 
        len(series) - 1, 
        loc=mean, 
        scale=sem
    )
#     print(mean, sem, confidence_interval)
    return confidence_interval[0]


all_hpars = ['model_type'] + [
    item 
    for sublist in list(type_to_hpars.values()) 
    for item in sublist
]

subset = df[(df['preprocessing.num_gesture_classes'] == '51')]
# print(subset.shape)
gb = subset.groupby(all_hpars, dropna=False)
subset['val.macro avg.f1-score.count'] = (
    gb['val.macro avg.f1-score']
    .transform('count')
)

subset = subset[
    subset['val.macro avg.f1-score.count'].between(5, 100)
]

subset['val.macro avg.f1-score.tenth_conf_interval'] = (
    gb['val.macro avg.f1-score']
    .transform(tenth_conf_interval)
)
subset['val.macro avg.f1-score.mean']  = gb['val.macro avg.f1-score'].transform('mean')
subset['val.macro avg.f1-score.min']   = gb['val.macro avg.f1-score'].transform('min')
subset['val.macro avg.f1-score.max']   = gb['val.macro avg.f1-score'].transform('max')
subset['val.macro avg.f1-score.std']   = gb['val.macro avg.f1-score'].transform('std')
subset['val.macro avg.f1-score.count'] = gb['val.macro avg.f1-score'].transform('count')

subset['group_idx'] = gb.ngroup()
subset = subset.sort_values('val.macro avg.f1-score.tenth_conf_interval')

subset

### Plot models grouped by hyperparameters

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(40, 8))

order = subset.sort_values(
    'val.macro avg.f1-score.tenth_conf_interval'
)['group_idx'].unique()

sns.stripplot(
    data=subset,
    y='val.macro avg.f1-score',
    x='group_idx',
    hue='model_type',
    order=order,
    size=5,
    alpha=0.5,
    ax=ax,
)

sns.pointplot(
    data=subset, 
    x="group_idx", 
    y="val.macro avg.f1-score", 
    hue="model_type",
    linestyle="none", 
    errorbar=None,
    marker="_", 
    markersize=5, 
    palette='dark:black',
    markeredgewidth=1,
    zorder=10,
    ax=ax,
)

ax.set(
    title='$F_1$ score for each set of hyperparameters, by model type',
    xlabel='Hyperparameter index',
    ylabel='$F_1$-score',
    ylim=((-0.05, 1.05))
)
ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(
    order,
    rotation=90,
)

handles, labels = ax.get_legend_handles_labels()
ax.add_artist(plt.legend(handles[:-5], labels[:-5], title="Model Type"))

plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

data = subset[subset['val.macro avg.f1-score.tenth_conf_interval'] > 0.6]

order = data.sort_values(
    'val.macro avg.f1-score.tenth_conf_interval'
)['group_idx'].unique()

sns.stripplot(
    data=data,
    y='val.macro avg.f1-score',
    x='group_idx',
    hue='model_type',
    order=order,
    hue_order=list(model_colours.keys()),
    size=5,
    jitter=False,
    alpha=0.5,
    ax=ax,
)

sns.pointplot(
    data=data, 
    x="group_idx", 
    y="val.macro avg.f1-score", 
    hue="model_type",
    linestyle="none", 
    errorbar=None,
    marker="_", 
    markersize=5, 
    palette='dark:black',
    markeredgewidth=1,
    zorder=10,
    ax=ax,
)

ax.set(
    title='$F_1$ score for each set of hyperparameters\n(Lower bound of the 90% CI $>$ 0.6)',
    xlabel='Hyperparameter index',
    ylabel='$F_1$-score',
)
ax.yaxis.grid(True)
ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(
    order,
    rotation=90,
)

handles, labels = ax.get_legend_handles_labels()
ax.add_artist(plt.legend(handles[:5], labels[:5], title="Model Type"))

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_best_hpar_comparison.pdf',
    bbox_inches='tight',
)
plt.show()

In [ ]:
model_type = 'FFNN'

def prettify_col(col):
    return {
        'val.macro avg.f1-score.mean': '$F_1$-score Mean',
        'val.macro avg.f1-score.std': '$F_1$-score Std.Dev.',
        'group_idx': 'Index',
        'cusum.thresh': 'Threshold',
        'ffnn.dropout_rate': 'Dropout Rate',
        'ffnn.l2_coefficient': 'L2 Coefficient',
        'ffnn.l2_coefficient.log10': '$\log_{10}(\text{L2 Coefficient})$',
        'ffnn.nodes_per_layer.1': 'Nodes (layer 1)',
        'ffnn.nodes_per_layer.2': 'Nodes (layer 2)',
        'ffnn.nodes_per_layer.3': 'Nodes (layer 3)',
        'hffnn.majority.ffnn.dropout_rate': 'Majority: Dropout Rate',
        'hffnn.majority.ffnn.l2_coefficient': 'Majority: L2 Coefficient',
        'hffnn.majority.ffnn.nodes_per_layer.1': 'Majority: Nodes (layer 1)',
        'hffnn.majority.ffnn.nodes_per_layer.2': 'Majority: Nodes (layer 2)',
        'hffnn.majority.ffnn.nodes_per_layer.3': 'Majority: Nodes (layer 3)',
        'hffnn.majority.nn.batch_size': 'Majority: Batch Size',
        'hffnn.majority.nn.epochs': 'Majority: Epochs',
        'hffnn.majority.nn.learning_rate': 'Majority: Learning Rate',
        'hffnn.majority.nn.optimizer': 'Majority: Optimizer',
        'hffnn.minority.ffnn.dropout_rate': 'Minority: Dropout Rate',
        'hffnn.minority.ffnn.l2_coefficient': 'Minority: L2 Coefficient',
        'hffnn.minority.ffnn.nodes_per_layer.1': 'Minority: Nodes (layer 1)',
        'hffnn.minority.ffnn.nodes_per_layer.2': 'Minority: Nodes (layer 2)',
        'hffnn.minority.ffnn.nodes_per_layer.3': 'Minority: Nodes (layer 3)',
        'hffnn.minority.nn.batch_size': 'Minority: Batch Size',
        'hffnn.minority.nn.epochs': 'Minority: Epochs',
        'hffnn.minority.nn.learning_rate': 'Minority: Learning Rate',
        'hffnn.minority.nn.optimizer': 'Minority: Optimizer',
        'hmm.covariance_type': 'Covariance Type',
        'nn.batch_size': 'Batch Size',
        'nn.batch_size.log10': '$\log_{10}(\text{Batch Size})$',
        'nn.epochs': 'Epochs',
        'nn.learning_rate': 'Learning Rate',
        'nn.learning_rate.log10': '$\log_{10}(\text{Learning Rate})$',
        'nn.optimizer': 'Optimizer',
        'svm.c': 'C',
        'svm.class_weight': 'Class Weight',
    }.get(col, col)

def df_to_latex(df, model_type):
    path = f'../../report/src/tables/05_best_{model_type.lower().replace(" ", "_")}_hpars.generated.tex'
    print('DONT FORGET TO UPDATE LaTeX tables: ', path)
    df.to_latex(
        path,
        caption=f'Top {len(df)} performing {model_type} hyperparameter combinations, ordered by '
                f'the lower bound of the 90 percent confidence interval for $F_1$-score.',
        label=f'tab:05_best_{model_type.lower().replace(" ", "_")}_hpars',
        index=False,
        float_format=lambda x: '%.3e' % x,
        na_rep='-'
    )

for model_type in type_to_hpars.keys():
    latex_df = (
        subset[
            subset['model_type'] == model_type
        ]
        .sort_values('val.macro avg.f1-score.tenth_conf_interval', ascending=False)
        .groupby('group_idx')
        .tail(1)
        [
            ['group_idx', 'val.macro avg.f1-score.mean', 'val.macro avg.f1-score.std'] 
            + type_to_hpars[model_type]
        ]
        .head(10)
        .reset_index(drop=True)
        .rename(columns=prettify_col)
        .replace({
            'tied': 'Tied',
            'spherical': 'Spherical',
            'diag': 'Diagonal',
            'full': 'Full',
            'balanced': 'Balanced',
        } | {} if model_type != 'SVM' else {
            np.nan: 'Unbalanced',
        } | {} if model_type not in ('HFFNN', 'FFNN') else {
            np.nan: 'None',
        })
    )
    display(latex_df)
    
    if model_type == 'HFFNN':
        majority = latex_df[[c for c in latex_df.columns if 'Minority' not in c]]
        minority = latex_df[[c for c in latex_df.columns if 'Majority' not in c]]
        df_to_latex(majority, 'Majority HFFNN')
        df_to_latex(minority, 'Minority HFFNN')
    else:
        df_to_latex(latex_df, model_type)
    

### Model types grouped by hpar index

In [ ]:
from matplotlib.gridspec import GridSpec
fig = plt.figure(figsize=(10, 10))
gs = GridSpec(4, 2, figure=fig)
ax1 = fig.add_subplot(gs[0, :])
ax2 = fig.add_subplot(gs[1, :])
ax3 = fig.add_subplot(gs[2, :])
ax4 = fig.add_subplot(gs[3, 0])
ax5 = fig.add_subplot(gs[3, 1])
# axs = np.array([ax1, ax2, ax3, ax4, ax5])
model_axs = {
    'FFNN':  ax1,
    'HFFNN': ax2,
    'SVM':   ax3,
    'HMM':   ax4,
    'CuSUM': ax5,
}

for model_type, color in model_colours.items():
    ax = model_axs[model_type]
    data = subset[subset['model_type'] == model_type]
    
    order = data.sort_values(
        'val.macro avg.f1-score.tenth_conf_interval'
    )['group_idx'].unique()

    sns.stripplot(
        data=data,
        y='val.macro avg.f1-score',
        x='group_idx',
#         hue='model_type',
        color=color,
        order=order,
        size=5,
        alpha=0.5,
        ax=ax,
        jitter=False,
        legend=False,
    )

    sns.pointplot(
        data=data, 
        x="group_idx", 
        y="val.macro avg.f1-score", 
        hue="model_type",
        linestyle="none", 
        errorbar=None,
        marker="_", 
        markersize=5, 
        palette='dark:black',
        markeredgewidth=1,
        zorder=10,
        ax=ax,
        legend=False,
    )
    
    ax.set(
        title=f'$F_1$-score for each set of {model_type} hyperparameters',
        xlabel='Hyperparameter index',
        ylabel='$F_1$-score',
        ylim=((-0.05, 1.05)),
    )
    ax.yaxis.grid(True)
    ax.set_xticks(ax.get_xticks())
    ax.set_xticklabels(order, rotation=90)

plt.tight_layout()
plt.savefig(
    '../../report/src/imgs/graphs/05_hpar_comparison_per_model_type.pdf',
    bbox_inches='tight',
)
plt.show()

## Top X performing models by precision/recall/F1

In [ ]:
%%script false --no-raise-error
# NOTE: Since every model gets 5 repetitions, the "top 5" will likely only contain one model.
top_n = 9
metric = 'val.macro avg.f1-score'

data = df[
    (df['preprocessing.num_gesture_classes'] == '51')
].sort_values(metric, ascending=False).head(top_n)

fig, axs = plt.subplots(3, 3, figsize=(10, 10))

for i, ax in enumerate(axs.flatten()):
    best = data.iloc[i]
    show_conf_mat_from_model(f"../{best['model_dir']}", ax)
    ax.set(
        title=f"{best['model_type']}\n$F_1=${np.round(best['val.macro avg.f1-score'], 4)}",
    )
plt.tight_layout()

## Get statistics for each hyperparameter combo

In [ ]:
data = df[
    (df['model_type'] == 'FFNN') & 
    (df['preprocessing.num_gesture_classes'] == '51') &
    (df['preprocessing.seed'] == 42.0)
]

data[[c for c in subset_cols if 'ffnn.' in c or 'nn.' in c]].head(31)
data[subset_cols].head(31)

## ECDF for all model types

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 6))
for model_type, color in model_colours.items():
    scores = df.loc[
        (df['preprocessing.num_gesture_classes'] == '51')
        & (df['model_type'] == model_type),
        'val.macro avg.f1-score'
    ]
    sorted_scores = np.sort(scores)
    ecdf = np.arange(1, len(sorted_scores) + 1) / len(sorted_scores)
    ax.plot(sorted_scores, ecdf, color=color, label=model_type)
plt.legend()
ax.set(
    title='ECDF for all model types',
    xlabel='$F_1$-score',
    ylabel='Cumulative Probability',
)
plt.show()

## Training vs Inference times

In [ ]:
# Three plots showing the precision and recall for all models.
# Each plot showing either 51, 50, or 5 gesture classes
fig, axs = plt.subplots(2, 3, figsize=(12, 8))

sns.scatterplot(
    data=df[df['preprocessing.num_gesture_classes'] == '51'],
    x='val.pred_time_per_obs',
    y='trn.pred_time_per_obs',
    s=10,
    alpha=0.5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    ax=axs[0, 0],
)

sns.scatterplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
        & (df['trn.pred_time_per_obs'] < 0.005)
    ],
    x='val.pred_time_per_obs',
    y='trn.pred_time_per_obs',
    s=10,
    alpha=0.5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    ax=axs[0, 1],
)

sns.scatterplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
        & (df['trn.pred_time_per_obs'] < 0.00005)
    ],
    x='val.pred_time_per_obs',
    y='trn.pred_time_per_obs',
    s=10,
    alpha=0.5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    ax=axs[0, 2],
)


sns.scatterplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
    ],
    x='fit_time_per_obs',
    y='trn.pred_time_per_obs',
    s=10,
    alpha=0.5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    ax=axs[1, 0],
)

sns.scatterplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
        & (df['trn.pred_time_per_obs'] < 0.005)
    ],
    x='fit_time_per_obs',
    y='trn.pred_time_per_obs',
    s=10,
    alpha=0.5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    ax=axs[1, 1],
)

sns.scatterplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
        & (df['trn.pred_time_per_obs'] < 0.00005)
    ],
    x='fit_time_per_obs',
    y='trn.pred_time_per_obs',
    s=10,
    alpha=0.5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    ax=axs[1, 2],
)

axs[0, 0].plot([0, 0.1], [0, 0.1], color='black', alpha=.1)
axs[0, 1].plot([0, 0.005], [0, 0.005], color='black', alpha=.1)
axs[0, 2].plot([0, 0.00005], [0, 0.00005], color='black', alpha=.1)

axs[0, 0].set(
    title='a) Inference times (seconds per observation)\n',
    xlabel='Inference time per observation (validation)',
    ylabel='Inference time per observation (training)',
)
axs[0, 1].set(
    title='b) Inference times (seconds per observation)\n(training time $< 0.005$s)',
    xlabel='Inference time per observation (validation)',
    ylabel='Inference time per observation (training)',
)
axs[0, 2].set(
    title='c) Inference times (seconds per observation)\n(training time $< 0.00005$s)',
    xlabel='Inference time per observation (validation)',
    ylabel='Inference time per observation (training)',
)

axs[1, 0].set(
    title='d) Inference vs training times (seconds per observation)\n',
    xlabel='Training time per observation',
    ylabel='Inference time per observation (training)',
)
axs[1, 1].set(
    title='e) Inference vs training times (seconds per observation)\n(training time $< 0.005$s)',
    xlabel='Inference time per observation',
    ylabel='Inference time per observation (training)',
)
axs[1, 2].set(
    title='f) Inference vs training times (seconds per observation)\n(training time $< 0.00005$s)',
    xlabel='Training time per observation',
    ylabel='Inference time per observation (training)',
)

for ax in axs.flatten():
    ax.legend().set_title("Model Type")

plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_inf_trn_times_per_obs.pdf',
    bbox_inches='tight'
)

## Inference time vs $F_1$

In [ ]:
# Three plots showing the precision and recall for all models.
# Each plot showing either 51, 50, or 5 gesture classes
fig, axs = plt.subplots(1, 2, figsize=(8, 4))

sns.scatterplot(
    data=df[df['preprocessing.num_gesture_classes'] == '51'],
    x='val.macro avg.f1-score',
    y='val.pred_time_per_obs',
    s=10,
    alpha=0.5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    ax=axs[0],
)
axs[0].set_ylim((-0.001, 0.011))

sns.scatterplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51') &
        (df['val.pred_time_per_obs'] < 0.0001)
    ],
    x='val.macro avg.f1-score',
    y='val.pred_time_per_obs',
    s=10,
    alpha=0.5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    ax=axs[1],
)
plt.tight_layout()
axs[0].set_xlim((-0.05, 1.05))
axs[1].set_xlim((-0.05, 1.05))
axs[0].set_xlabel('$F_1$-score')
axs[1].set_xlabel('$F_1$-score')
axs[0].set_ylabel('Inference time per observation (s)')
axs[1].set_ylabel('Inference time per observation (s)')
axs[0].set_title('Inference time per observation against $F_1$ score\n')
axs[1].set_title('Inference time per observation against $F_1$ score\n(0 to 0.0001s)')

axs[0].legend().set_title("Model Type")
axs[1].legend().set_title("Model Type")

plt.savefig(
    '../../report/src/imgs/graphs/05_inference_time_per_obs_vs_f1.pdf', 
    bbox_inches='tight'
)

## Confusion Matrices of the best models

In [ ]:
ngestures = sorted(df['preprocessing.num_gesture_classes'].unique())
model_types = sorted(df['model_type'].unique())

fig, axs = plt.subplots(
    len(ngestures),
    len(model_types),
    figsize=(len(model_types)*6, len(ngestures)*6),
    squeeze=False
)

for i, ngesture in enumerate(ngestures):
    for j, model_type in enumerate(model_types):
        best = df[
            (df['preprocessing.num_gesture_classes'] == ngesture) &
            (df['model_type'] == model_type)
        ].sort_values('val.macro avg.f1-score', ascending=False)
        if len(best) == 0:
            axs[i,j].axis('off')
            continue
        best = best.iloc[0]
        print(ngesture, model_type, best['model_dir'])
        try:
            show_conf_mat_from_model(f"../{best['model_dir']}", axs[i, j])
        except FileNotFoundError:
            show_conf_mat_from_model(f"./{best['model_dir']}", axs[i, j])
        axs[i, j].set(
            title=f"Best {model_type}: {ngesture} gestures\n($F_1=${np.round(best['val.macro avg.f1-score'], 4)})",
        )

plt.tight_layout()

## Mean Confidence Matrices

### 5 Gesture classes

In [ ]:
nclasses = '5'
data = df[
    (df['preprocessing.num_gesture_classes'] == nclasses)
]
conf_mats = {}
conf_mat_totals = {}

hpar = 'model_type'

assert len(data[hpar].unique()) < 10

for i, row in data.iterrows():
    y_true, y_pred = get_npz_data_from_model('../' + row['model_dir'])
    cm = tf.math.confusion_matrix(
        y_true.flatten(), 
        y_pred.flatten()
    ).numpy()
    f1_score = sklearn.metrics.f1_score(
        y_true.flatten(), 
        y_pred.flatten(),
        average='macro',
        zero_division=0,
    )
    hpar_item = row[hpar]
    
    if hpar_item in conf_mats:
        conf_mats[hpar_item] += cm.astype(float) * f1_score
        conf_mat_totals[hpar_item] += f1_score
    else:
        conf_mats[hpar_item] = cm.astype(float) * f1_score
        conf_mat_totals[hpar_item] = f1_score

fig, axs = plt.subplots(
    2,2,
    figsize=(8, 8)
)
print(conf_mats.keys())
axs = axs.flatten()
for i, (hpar_item, conf_mat) in enumerate(conf_mats.items()):
#     conf_mat[-1, -1] = 0
#     conf_mat /= conf_mat_totals[hpar_item]
    conf_mat /= conf_mat.max()
    print(conf_mat_totals[hpar_item])
    vis.conf_mat(conf_mat, ax=axs[i], norm=None)
    axs[i].set_title(
        f'{hpar_item} Confusion Matrices\n(weighted mean, {nclasses} classes)'
    )

plt.tight_layout()
plt.savefig(
    f'../../report/src/imgs/graphs/05_mean_conf_mat_{nclasses}_classes.pdf',
    bbox_inches='tight',
)

### 50 Gesture classes

In [ ]:
nclasses = '50'
data = df[
    (df['preprocessing.num_gesture_classes'] == nclasses)
]
conf_mats = {}
conf_mat_totals = {}

hpar = 'model_type'

assert len(data[hpar].unique()) < 10

for i, row in data.iterrows():
    y_true, y_pred = get_npz_data_from_model('../' + row['model_dir'])
    cm = tf.math.confusion_matrix(
        y_true.flatten(), 
        y_pred.flatten()
    ).numpy()
    f1_score = sklearn.metrics.f1_score(
        y_true.flatten(), 
        y_pred.flatten(),
        average='macro',
        zero_division=0,
    )
    hpar_item = row[hpar]
    
    if hpar_item in conf_mats:
        conf_mats[hpar_item] += cm.astype(float) * f1_score
        conf_mat_totals[hpar_item] += f1_score
    else:
        conf_mats[hpar_item] = cm.astype(float) * f1_score
        conf_mat_totals[hpar_item] = f1_score

fig, axs = plt.subplots(
    2, 2,
    figsize=(8, 8)
)
print(conf_mats.keys())
axs = axs.flatten()
for i, (hpar_item, conf_mat) in enumerate(conf_mats.items()):
    conf_mat /= conf_mat.max()
    vis.conf_mat(conf_mat, ax=axs[i], norm=None)
    axs[i].set_title(
        f'{hpar_item} Confusion Matrices\n(weighted mean, {nclasses} classes)'
    )

plt.tight_layout()
plt.savefig(
    f'../../report/src/imgs/graphs/05_mean_conf_mat_{nclasses}_classes.pdf',
    bbox_inches='tight',
)

### 51 Gesture classes

In [ ]:
nclasses = '51'
data = df[
    (df['preprocessing.num_gesture_classes'] == nclasses)
]
conf_mats = {}
conf_mat_totals = {}

hpar = 'model_type'

assert len(data[hpar].unique()) < 10

for i, row in data.iterrows():
    try:
        y_true, y_pred = get_npz_data_from_model('../' + row['model_dir'])
    except FileNotFoundError:
        y_true, y_pred = get_npz_data_from_model('./' + row['model_dir'])
    cm = tf.math.confusion_matrix(
        y_true.flatten(), 
        y_pred.flatten()
    ).numpy()
    f1_score = sklearn.metrics.f1_score(
        y_true.flatten(), 
        y_pred.flatten(),
        average='macro',
        zero_division=0,
    )
    hpar_item = row[hpar]
    
    if hpar_item in conf_mats:
        conf_mats[hpar_item] += cm.astype(float) * f1_score
        conf_mat_totals[hpar_item] += f1_score
    else:
        conf_mats[hpar_item] = cm.astype(float) * f1_score
        conf_mat_totals[hpar_item] = f1_score
    if i % 100 == 0:
        print(i, end=' ', flush=True)
        
fig, axs = plt.subplots(
    2, 3,
    figsize=(12, 8)
)
axs = axs.flatten()
for i, (hpar_item, conf_mat) in enumerate(conf_mats.items()):
    conf_mat[-1, -1] = 0
    conf_mat /= conf_mat.max()
    vis.conf_mat(conf_mat, ax=axs[i], norm=None)
    axs[i].set_title(
        f'{hpar_item} Confusion Matrices\n(weighted mean, {nclasses} classes)'
    )
axs[-1].axis('off')

plt.tight_layout()
plt.savefig(
    f'../../report/src/imgs/graphs/05_mean_conf_mat_{nclasses}_classes.pdf',
    bbox_inches='tight',
)

In [ ]:
# nclasses = '51'
model_type = 'FFNN'
for model_type in ('FFNN', 'SVM', 'HFFNN', 'CuSUM', 'HMM'):
    data = df[
        (df['model_type'] == model_type)
    ]
    conf_mats = {}
    conf_mat_totals = {}

    hpar = 'preprocessing.num_gesture_classes'

    assert len(data[hpar].unique()) < 10
    print(model_type, flush=True)

    for i, row in data.sort_values(hpar).iterrows():
        try:
            y_true, y_pred = get_npz_data_from_model('../' + row['model_dir'])
        except FileNotFoundError:
            y_true, y_pred = get_npz_data_from_model('./' + row['model_dir'])
        cm = tf.math.confusion_matrix(
            y_true.flatten(), 
            y_pred.flatten()
        ).numpy()
        f1_score = sklearn.metrics.f1_score(
            y_true.flatten(), 
            y_pred.flatten(),
            average='macro',
            zero_division=0,
        )
        hpar_item = row[hpar]

        if hpar_item in conf_mats:
            conf_mats[hpar_item] += cm.astype(float) * f1_score
            conf_mat_totals[hpar_item] += f1_score
        else:
            conf_mats[hpar_item] = cm.astype(float) * f1_score
            conf_mat_totals[hpar_item] = f1_score
        if i % 100 == 0:
            print(i, end=' ', flush=True)

    fig, axs = plt.subplots(
        1, len(conf_mats.keys()),
        figsize=(4*len(conf_mats.keys()), 4),
        squeeze=False,
    )
    axs = axs.flatten()
    for i, (hpar_item, conf_mat) in enumerate(conf_mats.items()):
        if conf_mat.shape[0] == 51:
            conf_mat[-1, -1] = 0
        conf_mat /= conf_mat.max()
        vis.conf_mat(conf_mat, ax=axs[i], norm=None)
        axs[i].set_title(
            f'{model_type} {hpar_item}-class Confusion Matrices\n(weighted mean)'
        )

    plt.tight_layout()
    plt.savefig(
        f'../../report/src/imgs/graphs/05_mean_conf_mat_{model_type.lower()}.pdf',
        bbox_inches='tight',
    )
    plt.show()

## Regularization Plots

In [ ]:
n_gesture_classes = (
    '5',
    '50',
    '51',
)
fig, axs = plt.subplots(len(n_gesture_classes), 2, figsize=(6, len(n_gesture_classes)*3))

for i, ngestures in enumerate(n_gesture_classes):
    data = df[df['preprocessing.num_gesture_classes'] == ngestures]
    sns.scatterplot(
        data=data,
        x='ffnn.l2_coefficient',
        y='ratio.macro avg.f1-score',
        ax=axs[i, 0]
    )
    sns.scatterplot(
        data=data,
        x='ffnn.dropout_rate',
        y='ratio.macro avg.f1-score',
        ax=axs[i, 1]
    )
    axs[i, 0].set_title(f'{ngestures} gestures')
    axs[i, 1].set_title(f'{ngestures} gestures')

plt.suptitle("$F_1$-ratio against regularisation")
plt.tight_layout()

## Ratio $F_1$ scores vs actual $F_1$ scores

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(8, 8))


sns.scatterplot(
    data=df[df['preprocessing.num_gesture_classes'] == '51'],
    x='val.macro avg.f1-score',
    y='trn.macro avg.f1-score',
    s=5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    alpha=0.5,
    ax=axs[0, 0],
)
axs[0, 0].set_xlim((-0.05, 1.05))
axs[0, 0].set_ylim((-0.05, 1.05))
axs[0, 0].plot([0,1], [0,1], color='black', alpha=.1)
axs[0, 0].set_title("a) Training vs Validation $F_1$ score\n")
axs[0, 0].set_xlabel("Validation $F_1$")
axs[0, 0].set_ylabel("Training $F_1$")
axs[0, 0].legend().set_title('Model Type')

sns.scatterplot(
    data=df[df['preprocessing.num_gesture_classes'] == '51'],
    x='val.macro avg.f1-score',
    y='trn.macro avg.f1-score',
    s=5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    alpha=0.5,
    ax=axs[0, 1],
)
axs[0, 1].set_xlim((0.5, 1.05))
axs[0, 1].set_ylim((0.5, 1.05))
axs[0, 1].plot([0,1], [0,1], color='black', alpha=.1)
axs[0, 1].set_title("b) Training vs Validation $F_1$ score\n(magnified)")
axs[0, 1].set_xlabel("Validation $F_1$")
axs[0, 1].set_ylabel("Training $F_1$")
axs[0, 1].legend().set_title('Model Type')

sns.scatterplot(
    data=df[df['preprocessing.num_gesture_classes'] == '51'],
    x='val.macro avg.f1-score',
    y='ratio.macro avg.f1-score',
    s=5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    alpha=0.5,
    ax=axs[1, 0],
)
axs[1, 0].set_xlim((-0.05, 1.05))
axs[1, 0].set_title("c) $F_1$-ratio vs $F_1$-score\n")
axs[1, 0].set_xlabel("Validation $F_1$")
axs[1, 0].set_ylabel(r"$F_1$-ratio ($\frac{Training}{Validation}$)")
axs[1, 0].legend().set_title('Model Type')
axs[1, 0].plot([0,1], [1, 1], color='black', alpha=.1)

sns.scatterplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
        & (df['val.macro avg.f1-score'] >= 0.5)
    ],
    x='val.macro avg.f1-score',
    y='ratio.macro avg.f1-score',
    s=5,
    hue='model_type',
    hue_order=list(model_colours.keys()),
    alpha=0.5,
    ax=axs[1, 1],
)

axs[1, 1].set_xlim((0.5, 1.05))
axs[1, 1].set_title("d) $F_1$-ratio vs $F_1$-score\n(magnified)")
axs[1, 1].set_xlabel("Validation $F_1$")
axs[1, 1].set_ylabel(r"$F_1$-ratio ($\frac{Training}{Validation}$)")
axs[1, 1].legend().set_title('Model Type')
axs[1, 1].plot([0,1], [1, 1], color='black', alpha=.1)

plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_f1_vs_f1_ratio.pdf'
)


## Training/validation loss ratios

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(8, 4))

sns.scatterplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
        & (df['model_type'] == 'FFNN')
    ],
    y='val.loss',
    x='trn.loss',
    color='tab:blue',
    s=20,
    alpha=0.5,
    ax=axs[0],
)


sns.scatterplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
        & (df['model_type'] == 'FFNN')
    ].assign(**{
        'ratio.loss': lambda x: x['trn.loss'] / x['val.loss']
    }),
    x='ratio.loss',
    y='val.loss',
    color='tab:blue',
    s=20,
    alpha=0.5,
    ax=axs[1],
)


axs[0].set_ylim((-0.05, 3.6))
axs[1].set_ylim((-0.05, 3.6))

axs[1].plot([1, 1], [0, axs[1].get_ylim()[1]], color='black', alpha=.1)

min_max = min(axs[0].get_xlim()[1], axs[0].get_ylim()[1] )
axs[0].plot([0, min_max], [0, min_max], color='black', alpha=.1)

axs[0].set(
    title='Validation loss vs training loss\n(FFNN only)',
    xlabel='Training Loss',
    ylabel='Validation Loss',
)
axs[1].set(
    title='Validation loss vs loss ratio\n(FFNN only)',
    xlabel=r'Loss ratio ($\frac{Training}{Validation}$)',
    ylabel='Validation Loss',
)

plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_val_trn_loss_ratios.pdf'
)

print("TODO: The training and validation loss aren't comparable because the training loss is weighed but validation loss is not.")
print("TODO: also not comparable because of dropout")

## Precision vs Recall plots for each model individually

In [ ]:
for model_type, color in model_colours.items():
    data = df[
        (df['preprocessing.num_gesture_classes'] == '51')
        & (df['model_type'] == model_type)
    ]

    fig, ax = plt.subplots(1, 1, figsize=(6, 6))
    p_min = data['val.macro avg.precision'].min() / 1.10
    p_max = data['val.macro avg.precision'].max() * 1.10
    r_min = data['val.macro avg.recall'].min()    / 1.10
    r_max = data['val.macro avg.recall'].max()    * 1.10
    print(f"{p_min=}, {p_max=}, {r_min=}, {r_max=}, ")
    recall_grid, precision_grid = np.meshgrid(
        np.linspace(p_min, p_max, 100), 
        np.linspace(r_min, r_max, 100)
    )
    f1_score = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)

    print(f1_score.min())
    contours = ax.contour(
        recall_grid, 
        precision_grid,
        f1_score, 
        levels=np.linspace(
            f1_score.min(),
            f1_score.max(),
            10
        ), 
        colors='black',
        alpha=0.25
    )
    ax.clabel(contours, inline=True, fontsize=8, fmt='%.2f')

    sns.scatterplot(
        data=data,
        y='val.macro avg.recall',
        x='val.macro avg.precision',
        color=color,
        alpha=0.5,
        s=20,
        ax=ax,
    )

    p_range = p_max - p_min
    r_range = r_max - r_min
    ax.set(
        title=f"Precision vs Recall For {model_type} models\n($F_1$ contours in grey)",
        xlabel='Precision',
        ylabel='Recall',
        xlim=(p_min - p_range*0.025, p_max + p_range*0.025),
        ylim=(r_min - r_range*0.025, r_max + r_range*0.025),
    )
    plt.savefig(
        f'../../report/src/imgs/graphs/05_in_depth_{model_type.lower()}_p_vs_r.pdf',
        bbox_inches='tight',
    )
    plt.show()

## In depth FFNN plots

- Clusters in the recall of different models
- Hyperparameters vs f1 score/recall/precision
- No correlation with the inference time


In [ ]:
metric = (
    'f1-score',
    'precision',
    'recall',
)

metric_labels = (
    '$F_1$',
    'Precision',
    'Recall',
)

hyperpars = (
    'ffnn.dropout_rate',
    'ffnn.l2_coefficient.log10',
    'nn.batch_size.log10',
    'nn.learning_rate.log10',
    'ffnn.nodes_per_layer.1',
    'ffnn.nodes_per_layer.2',
    'ffnn.nodes_per_layer.3',
)
df[[
#     'ffnn.dropout_rate.log10',
    'ffnn.l2_coefficient.log10',
    'nn.batch_size.log10',
    'nn.learning_rate.log10',
]] = np.log10(df[[
#     'ffnn.dropout_rate',
    'ffnn.l2_coefficient',
    'nn.batch_size',
    'nn.learning_rate',
]])
xlabels = (
    'Dropout Rate',
    'L2 Coefficient ($\log_{10}$)',
    'Batch Size ($\log_{10}$)',
    'Learning Rate ($\log_{10}$)',
    'Nodes in Layer 1',
    'Nodes in Layer 2',
    'Nodes in Layer 3',
)

max_log10_val_loss = -0.25
data = df[
    (df['preprocessing.num_gesture_classes'] == '51') 
    & (df['model_type'] == 'FFNN')
    & (df['val.loss.log10'] <= max_log10_val_loss)
]


for metric, metric_label in zip(metrics, metric_labels):
    fig, axs = plt.subplots(
        4, 2,
        figsize=(12, 24),
        dpi=100,
    )
    for i, ax in enumerate(axs.flatten()):
        if len(hyperpars) <= i:
            ax.axis('off')
            continue

        sns.scatterplot(
            data=data,
            x=hyperpars[i],
            y=f'val.macro avg.{metric}',
            hue='val.loss.log10',
            palette=palette,
            s=20,
            ax=ax,
        )
        title_xlabel = xlabels[i].replace(" ($\log_{10}$)", "")
        ax.set(
            title=f'{metric_label} vs {title_xlabel}\n'
                  f'$\log_{{10}}($ Validation loss $) < {max_log10_val_loss}$',
            xlabel=xlabels[i],
            ylabel=metric_label,
            ylim=(-0.1, 1.1),
        )
        ax.legend().set_title('Validation Loss\n($\log_{10}$)')
    plt.tight_layout()
    plt.savefig(
        f'../../report/src/imgs/graphs/05_in_depth_ffnn_hpars_vs_{metric}.pdf',
        bbox_inches='tight',
    )
    plt.show()

In [ ]:
hyperpars = (
    'ffnn.dropout_rate',
    'ffnn.l2_coefficient.log10',
    'nn.batch_size.log10',
    'nn.learning_rate.log10',
    'ffnn.nodes_per_layer.1',
    'ffnn.nodes_per_layer.2',
    'ffnn.nodes_per_layer.3',
)

fig, axs = plt.subplots(3, 3, figsize=(12, 12))

for hpar, ax in zip(hyperpars, axs.flatten()):
    sns.scatterplot(
        data=df[
            (df['preprocessing.num_gesture_classes'] == '51') &
            (df['model_type'] == 'FFNN')
        ],
        x=hpar,
        y='val.loss.log10',
        hue='trn.loss.log10',
        ax=ax,
        palette='Spectral',
        s=10,
        alpha=0.5,
    )
    ax.legend().set_title('Training Loss\n($\log_{10}$)')
# plt.show()

sns.scatterplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51') &
        (df['model_type'] == 'FFNN')
    ],
    y='val.loss.log10',
    x='trn.loss.log10',
    ax=axs[-1, -1],
)
plt.tight_layout()

## In depth HFFNN plots



In [ ]:
# %%script false --no-raise-error 
# This cell isn't used

maj_min = (
    'majority',
    'minority',
)
for submodel in maj_min:
    hyperpars = (
        f'hffnn.{submodel}.ffnn.dropout_rate',
        f'hffnn.{submodel}.ffnn.l2_coefficient.log10',
        f'hffnn.{submodel}.nn.batch_size.log10',
        f'hffnn.{submodel}.nn.learning_rate.log10',
        f'hffnn.{submodel}.ffnn.nodes_per_layer.1',
        f'hffnn.{submodel}.ffnn.nodes_per_layer.2',
        f'hffnn.{submodel}.ffnn.nodes_per_layer.3',
    )
    df[[
        f'hffnn.{submodel}.ffnn.l2_coefficient.log10',
        f'hffnn.{submodel}.nn.batch_size.log10',
        f'hffnn.{submodel}.nn.learning_rate.log10',
    ]] = np.log10(df[[
        f'hffnn.{submodel}.ffnn.l2_coefficient',
        f'hffnn.{submodel}.nn.batch_size',
        f'hffnn.{submodel}.nn.learning_rate',
    ]])
    xlabels = (
        f'{submodel.title()} Dropout Rate',
        f'{submodel.title()} L2 Coefficient ($\log_{{10}}$)',
        f'{submodel.title()} Batch Size ($\log_{{10}}$)',
        f'{submodel.title()} Learning Rate ($\log_{{10}}$)',
        f'{submodel.title()} Nodes in Layer 1',
        f'{submodel.title()} Nodes in Layer 2',
        f'{submodel.title()} Nodes in Layer 3',
    )

    data = df[
        (df['preprocessing.num_gesture_classes'] == '51') &
        (df['model_type'] == 'HFFNN')
    ]

    fig, axs = plt.subplots(
        3, 3,
        figsize=(8, 8)
    )
    print(axs.shape)
    for i, ax in enumerate(axs.flatten()):
        if len(hyperpars) <= i:
            ax.axis('off')
            continue

        sns.scatterplot(
            data=data,
            x=hyperpars[i],
            y='val.macro avg.f1-score',
            s=10,
            ax=ax,
            color='tab:orange',
        )
        title_xlabel = xlabels[i].replace(" ($\log_{10}$)", "")
        ax.set(
            title=f'$F_1$ score vs {title_xlabel}',
            xlabel=xlabels[i],
            ylabel='$F_1$ score',
            ylim=(-0.1, 1.1),
        )
    plt.tight_layout()
    plt.savefig(
        f'../../report/src/imgs/graphs/05_in_depth_hffnn_{submodel}_hpars.pdf',
        bbox_inches='tight',
    )
    plt.show()

## In depth CuSUM plots

In [ ]:
data = df[
    (df['preprocessing.num_gesture_classes'] == '51')
    & (df['model_type'] == 'CuSUM')
]

fig, axs = plt.subplots(1, 2, figsize=(12, 6))
p_min = data['val.macro avg.precision'].min() / 1.10
p_max = data['val.macro avg.precision'].max() * 1.10
r_min = data['val.macro avg.recall'].min()    / 1.10
r_max = data['val.macro avg.recall'].max()    * 1.10

recall_grid, precision_grid = np.meshgrid(
    np.linspace(p_min, p_max, 100), 
    np.linspace(r_min, r_max, 100)
)
f1_score = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)

contours = axs[0].contour(
    recall_grid, 
    precision_grid,
    f1_score, 
    levels=np.linspace(
        f1_score.min(),
        f1_score.max(),
        10,
    ), 
    colors='black',
    alpha=0.25
)
axs[0].clabel(contours, inline=True, fontsize=8, fmt='%.2f')

sns.scatterplot(
    data=data,
    y='val.macro avg.recall',
    x='val.macro avg.precision',
    hue='cusum.thresh',
    alpha=0.5,
    s=20,
    ax=axs[0],
    palette=palette,
)

sns.stripplot(
    data=data,
    x='cusum.thresh',
    y='val.macro avg.f1-score',
    alpha=0.5,
    s=5,
    ax=axs[1],
    jitter=0.5,
    color=model_colours['CuSUM'],
    native_scale=True,
)

p_range = p_max - p_min
r_range = r_max - r_min
axs[0].set(
    title=f"a) Precision vs Recall for CuSUM models\n($F_1$ contours in grey)",
    xlabel='Precision',
    ylabel='Recall',
    xlim=(p_min - p_range*0.025, p_max + p_range*0.025),
    ylim=(r_min - r_range*0.025, r_max + r_range*0.025),
)

axs[0].legend().set_title('Threshold')

axs[1].set(
    title='b) CuSUM Threshold vs $F_1$ score',
    xlabel='CuSUM Threshold',
    ylabel='$F_1$ score',
)

plt.savefig(
    f'../../report/src/imgs/graphs/05_in_depth_cusum_p_vs_r_thresh.pdf',
    bbox_inches='tight',
)
plt.show()

## In depth SVM plots

- Correlation with the training time per observation
- Correlation with the precision/recall/f1
- clusters in recall-precision space

In [ ]:
data = df[
    (df['preprocessing.num_gesture_classes'] == '51')
    & (df['model_type'] == 'SVM')
]

data['svm.c.log10'] = np.log10(data['svm.c'])
data['svm.class_weight'] = data['svm.class_weight'].fillna('unbalanced')

data = data.rename(columns={
    'svm.c.log10': '$\log_{10}(C)$', 
    'svm.class_weight': 'Class Weight'
})

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
p_min = data['val.macro avg.precision'].min() / 1.10
p_max = data['val.macro avg.precision'].max() * 1.10
r_min = data['val.macro avg.recall'].min()    / 1.10
r_max = data['val.macro avg.recall'].max()    * 1.10

recall_grid, precision_grid = np.meshgrid(
    np.linspace(p_min, p_max, 100), 
    np.linspace(r_min, r_max, 100)
)
f1_score = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)

contours = axs[0].contour(
    recall_grid, 
    precision_grid,
    f1_score, 
    levels=np.linspace(
        f1_score.min(),
        f1_score.max(),
        10
    ), 
    colors='black',
    alpha=0.25
)
axs[0].clabel(contours, inline=True, fontsize=8, fmt='%.2f')

sns.scatterplot(
    data=data,
    y='val.macro avg.recall',
    x='val.macro avg.precision',
    hue='$\log_{10}(C)$',
    style='Class Weight',
    alpha=0.5,
    s=20,
    ax=axs[0],
    palette=palette,
)


sns.scatterplot(
    data=data,
    x='$\log_{10}(C)$',
    y='val.macro avg.f1-score',
    style='Class Weight',
    alpha=0.75,
    s=20,
    ax=axs[1],
    color='tab:purple',
)

axs[0].set(
    title=f"Precision vs Recall for SVMs\n($F_1$ contours in grey)",
    xlabel='Precision',
    ylabel='Recall',
)
axs[1].set(
    title=f"SVM Regularization parameter vs $F_1$ score",
    xlabel='SVM Regularization parameter C ($\log_{10}$)',
    ylabel='$F_1$ score',
)

plt.tight_layout()
plt.savefig(
    f'../../report/src/imgs/graphs/05_in_depth_svm_p_vs_r_class_weight_C.pdf',
    bbox_inches='tight',
)
plt.show()

In [ ]:
data = df[
    (df['preprocessing.num_gesture_classes'] == '51')
    & (df['model_type'] == 'SVM')
]

data['svm.class_weight'] = data['svm.class_weight'].fillna('unbalanced')

conf_mats = {}
conf_mat_totals = {}

hpar = 'svm.class_weight'

assert len(data[hpar].unique()) < 10

for i, row in data.iterrows():
    y_true, y_pred = get_npz_data_from_model('../' + row['model_dir'])
    cm = tf.math.confusion_matrix(
        y_true.flatten(), 
        y_pred.flatten()
    ).numpy()
    hpar_item = row[hpar]
    
    if hpar_item in conf_mats:
        conf_mats[hpar_item] += cm.astype(float)
        conf_mat_totals[hpar_item] += 1.0
    else:
        conf_mats[hpar_item] = cm.astype(float)
        conf_mat_totals[hpar_item] = 1.0


fig, axs = plt.subplots(
    1, len(conf_mats), 
    figsize=(4*len(conf_mats), 4)
)


for i, (hpar_item, conf_mat) in enumerate(conf_mats.items()):
    conf_mat[-1, -1] = 0
    conf_mat /= conf_mat_totals[hpar_item]
    vis.conf_mat(conf_mat, ax=axs[i], norm=None)
    axs[i].set_title(
        f'{hpar_item.title()} SVMs\n(Mean of {int(conf_mat_totals[hpar_item])} Confusion Matrices)'
    )

plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_in_depth_svm_conf_mats_unbalanced.pdf',
    bbox_inches='tight',
)


In [ ]:
# fig, axs = plt.subplots(2, 3, figsize=(12, 8))

data = df[
    (df['preprocessing.num_gesture_classes'] == '51')
    & (df['model_type'] == 'SVM')
]

data['svm.c.log10'] = np.log10(data['svm.c'])
data['svm.class_weight'] = data['svm.class_weight'].fillna('unbalanced')

data = data.rename(columns={
    'svm.c.log10': '$\log_{10}(C)$', 
    'svm.class_weight': 'Class Weight'
})
fig, ax = plt.subplots(1, 1, figsize=(4, 4))

sns.scatterplot(
    data=data,
    y='fit_time_per_obs',
    x='$\log_{10}(C)$',
    s=10,
    alpha=0.75,
#     hue='',
    style='Class Weight',
    color='tab:purple',
    palette=palette,
    ax=ax,
)

ax.set(
    title='SVM hyperparameters against fit time',
    ylabel='Fit time (seconds per observation)',
    xlabel='$\log_{10}(C)$',
)
plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_svm_hpars_vs_fit_time.pdf',
    bbox_inches='tight'
)

## In depth HMM plots

- Clusters in recall-precision space
- 

In [ ]:
model_type = 'HMM'
color='tab:red'
    
data = df[
    (df['preprocessing.num_gesture_classes'] == '51')
    & (df['model_type'] == model_type)
]

data['hmm.covariance_type'] = data['hmm.covariance_type'].replace({
    'spherical': 'Spherical',
    'diag': 'Diagonal',
    'full': 'Full',
    'tied': 'Tied',
})

fig, ax = plt.subplots(1, 1, figsize=(6, 6))
p_min = data['val.macro avg.precision'].min() / 1.10
p_max = data['val.macro avg.precision'].max() * 1.10
r_min = data['val.macro avg.recall'].min()    / 1.10
r_max = data['val.macro avg.recall'].max()    * 1.10

recall_grid, precision_grid = np.meshgrid(
    np.linspace(p_min, p_max, 100), 
    np.linspace(r_min, r_max, 100)
)
f1_score = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)

contours = ax.contour(
    recall_grid, 
    precision_grid,
    f1_score, 
    levels=np.linspace(
        f1_score.min(),
        f1_score.max(),
        10
    ), 
    colors='black',
    alpha=0.25
)
ax.clabel(contours, inline=True, fontsize=8, fmt='%.2f')

sns.scatterplot(
    data=data,
    y='val.macro avg.recall',
    x='val.macro avg.precision',
    hue='hmm.covariance_type',
    alpha=0.5,
    s=20,
    ax=ax,
    palette=other_colours
)

p_range = p_max - p_min
r_range = r_max - r_min
ax.set(
    title=f"Precision vs Recall for {model_type} models\n($F_1$ contours in grey)",
    xlabel='Precision',
    ylabel='Recall',
    xlim=(p_min - p_range*0.025, p_max + p_range*0.025),
    ylim=(r_min - r_range*0.025, r_max + r_range*0.025),
)

ax.legend().set_title('Covariance Type')

# plt.savefig(
#     f'../../report/src/imgs/graphs/05_in_depth_hmm_p_vs_r_covar_type.pdf',
#     bbox_inches='tight',
# )
plt.show()

In [ ]:

fig, axs = plt.subplots(2, 2, figsize=(8, 8))

data = df[
    (df['preprocessing.num_gesture_classes'] == '51')
    & (df['model_type'] == 'HMM')
]

data['hmm.covariance_type'] = data['hmm.covariance_type'].replace({
    'spherical': 'Spherical',
    'diag': 'Diagonal',
    'full': 'Full',
    'tied': 'Tied',
})


sns.scatterplot(
    data=data,
    x='val.pred_time_per_obs',
    y='trn.pred_time_per_obs',
    s=10,
    alpha=0.5,
    hue='hmm.covariance_type',
    ax=axs[0, 0],
    palette=other_colours
)

sns.scatterplot(
    data=data,
    x='fit_time_per_obs',
    y='trn.pred_time_per_obs',
    s=10,
    alpha=0.5,
    hue='hmm.covariance_type',
    ax=axs[1, 0],
    palette=other_colours
)

sns.scatterplot(
    data=data[data['trn.pred_time_per_obs'] <= 0.02],
    x='val.pred_time_per_obs',
    y='trn.pred_time_per_obs',
    s=10,
    alpha=0.5,
    hue='hmm.covariance_type',
    ax=axs[0, 1],
    palette=other_colours
)

sns.scatterplot(
    data=data[data['trn.pred_time_per_obs'] <= 0.02],
    x='fit_time_per_obs',
    y='trn.pred_time_per_obs',
    s=10,
    alpha=0.5,
    hue='hmm.covariance_type',
    ax=axs[1, 1],
    palette=other_colours
)

for ax in axs.flatten():
    ax.legend().set_title('Covariance Type')

axs[0, 0].set(
    title='a) Inference time on the Training and Validation sets\n',
    xlabel='Validation inference time (seconds per observation)',
    ylabel='Training inference time (seconds per observation)',
)

axs[0, 1].set(
    title='b) Inference time on the Training and Validation sets\n($< 0.02$s)',
    xlabel='Validation inference time (seconds per observation)',
    ylabel='Training inference time (seconds per observation)',
)

axs[1, 0].set(
    title='c) Inference and Fitting times\n',
    ylabel='Inference time (seconds per observation)',
    xlabel='Fitting time (seconds per observation)',
)

axs[1, 1].set(
    title='d) Inference and Fitting times\n($< 0.02$s)',
    ylabel='Inference time (seconds per observation)',
    xlabel='Fitting time (seconds per observation)',
)

plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_in_depth_hmm_inf_trn_time.pdf',
    bbox_inches='tight'
)

In [ ]:
data = df[
    (df['preprocessing.num_gesture_classes'] == '51')
    & (df['model_type'] == 'HMM')
]
data['hmm.covariance_type'] = data['hmm.covariance_type'].replace({
    'spherical': 'Spherical',
    'diag': 'Diagonal',
    'full': 'Full',
    'tied': 'Tied',
})

conf_mats = {}
conf_mat_totals = {}
precisions = {}
recalls = {}
f1_scores = {}
y_preds_dict = {}
y_true_dict = {}
reports_dict = {}

hpar = 'hmm.covariance_type'

assert len(data[hpar].unique()) < 10

for i, row in data.iterrows():
    y_true, y_pred = get_npz_data_from_model('../' + row['model_dir'])
    precision, recall, f1_score, _support = sklearn.metrics.precision_recall_fscore_support(
        y_true.flatten(), 
        y_pred.flatten(),
#         average='macro',
        zero_division=0,
    )
    
    cm = tf.math.confusion_matrix(
        y_true.flatten(), 
        y_pred.flatten()
    ).numpy()
    hpar_item = row[hpar]
    
    if hpar_item in conf_mats:
        conf_mats[hpar_item] += cm.astype(float)
        conf_mat_totals[hpar_item] += 1.0
        precisions[hpar_item] += precision
        recalls[hpar_item] += recall
        f1_scores[hpar_item] += f1_score
        y_preds_dict[hpar_item] = np.concatenate((y_preds_dict[hpar_item], y_pred.flatten()))
        y_true_dict[hpar_item] = np.concatenate((y_true_dict[hpar_item], y_true.flatten()))
    else:
        conf_mats[hpar_item] = cm.astype(float)
        conf_mat_totals[hpar_item] = 1.0
        precisions[hpar_item] = precision
        recalls[hpar_item] = recall
        f1_scores[hpar_item] = f1_score
        y_preds_dict[hpar_item] = np.copy(y_pred.flatten())
        y_true_dict[hpar_item] = np.copy(y_true.flatten())

fig, axs = plt.subplots(
    2, 2,
    figsize=(8, 8),
    squeeze=False,
)
axs = axs.flatten()
for i, (hpar_item, conf_mat) in enumerate(conf_mats.items()):
    vis.conf_mat(conf_mat / conf_mat.max(), ax=axs[i], norm=None)
    axs[i].set_title(
        f'{hpar_item.title()} HMMs\n'
        f'(Mean of {int(conf_mat_totals[hpar_item])} confusion matrices)'
    )

plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_in_depth_hmm_conf_mats_cov_type.pdf',
    bbox_inches='tight',
)
plt.show()
fig, axs = plt.subplots(
    2, 2,
    figsize=(8, 2),
)
axs = axs.flatten()
for i, (hpar_item, conf_mat) in enumerate(conf_mats.items()):
    vis.precision_recall_f1(
        precision=precisions[hpar_item] / conf_mat_totals[hpar_item],
        recall=recalls[hpar_item] / conf_mat_totals[hpar_item],
        f1=f1_scores[hpar_item] / conf_mat_totals[hpar_item],
        ax=axs[i],
    )
    axs[i].set_title(hpar_item)
plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_in_depth_hmm_prf1_plots_conv_type.pdf',
    bbox_inches='tight',
)
plt.show()

## FFNN Plots

### Heatmap-based pairplot of FFNN Hyperparameters

In [ ]:
data = df[
    (df['model_type'] == 'FFNN') &
    (df['preprocessing.num_gesture_classes'] == '51')
]

hpars_scale = [
    ('nn.learning_rate',       'log10'),
    ('ffnn.nodes_per_layer.1', 'log10'),
    ('ffnn.l2_coefficient',    'log10'),
    ('ffnn.dropout_rate',      'linear'),
]
# x_var_idx = 0
# y_var_idx = 1
def contour_nicely(x, y, z, xlabel, ylabel, xscale, yscale, fig, ax, levels=8):

    ax.tricontour(x, y, z, levels=levels, linewidths=0.25, colors='k')
    cntr2 = ax.tricontourf(x, y, z, levels=levels, cmap=palette)

    fig.colorbar(cntr2, ax=ax)
    ax.scatter(x, y, color='white', s=1)

    ax.set_xlabel(f'{xlabel} ({xscale})')
    if xscale == 'log10':
        ax.set_xticks(ax.get_xticks())
        ax.set_xticklabels([f'{np.power(10, t):.3g}' for t in ax.get_xticks()])

    ax.set_ylabel(f'{ylabel} ({yscale})')
    if yscale == 'log10':
        ax.set_yticks(ax.get_yticks())
        ax.set_yticklabels([f'{np.power(10, t):.3g}' for t in ax.get_yticks()])

    ax.set_title(f'{xlabel} vs {ylabel}')

    
fig, axs = plt.subplots(
    len(hpars_scale), 
    len(hpars_scale), 
    dpi=200, 
    squeeze=False,
    figsize=(20,20)
)
z = data['val.macro avg.f1-score'].values
for x_var_idx in range(len(hpars_scale)):
    for y_var_idx in range(x_var_idx+1, len(hpars_scale)):
    
        x = data[hpars_scale[x_var_idx][0]].values
        if hpars_scale[x_var_idx][1] == 'log10':
            x = np.log10(x)
        elif hpars_scale[x_var_idx][1] != 'linear':
            raise NotImplementedError(f"Scale {hpars_scale[x_var_idx][0]} is not implemented")
        y = data[hpars_scale[y_var_idx][0]].values
        if hpars_scale[y_var_idx][1] == 'log10':
            y = np.log10(y)
        elif hpars_scale[y_var_idx][1] != 'linear':
            raise NotImplementedError(f"Scale {hpars_scale[y_var_idx][0]} is not implemented")
            
        contour_nicely(
            y, x, z,
            hpars_scale[y_var_idx][0],
            hpars_scale[x_var_idx][0],
            hpars_scale[y_var_idx][1],
            hpars_scale[x_var_idx][1],
            fig, axs[x_var_idx, y_var_idx]
        )
plt.tight_layout()
plt.show()


## Plot inference/training times

In [ ]:
sns.stripplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
    ].assign(**{
        'fit_time_per_obs': lambda x: x['fit_time'] / x['trn.num_observations']
    }),
    x='model_type',
    y='fit_time_per_obs',
    hue='model_type',
    alpha=0.5,
    s=3,
    hue_order=list(model_colours.keys()),
    order=list(model_colours.keys()),
)
plt.show()
sns.stripplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
    ].assign(**{
        'trn.pred_time_per_obs': lambda x: x['trn.pred_time'] / x['trn.num_observations']
    }),
    x='model_type',
    y='trn.pred_time_per_obs',
    hue='model_type',
    alpha=0.5,
    s=3,
    hue_order=list(model_colours.keys()),
    order=list(model_colours.keys()),
)
plt.show()
sns.stripplot(
    data=df[
        (df['preprocessing.num_gesture_classes'] == '51')
    ].assign(**{
        'val.pred_time_per_obs': lambda x: x['val.pred_time'] / x['val.num_observations']
    }),
    x='model_type',
    y='val.pred_time_per_obs',
    hue='model_type',
    alpha=0.5,
    s=3,
    hue_order=list(model_colours.keys()),
    order=list(model_colours.keys()),
)
plt.show()

## Inference times vs num classes

In [ ]:
df['val.pred_time_per_obs'] = df['val.pred_time'] / df['val.num_observations']
df['val.pred_time_per_obs.log10'] = np.log10(df['val.pred_time_per_obs'].fillna(0))

fig, axs = plt.subplots(2, 2, figsize=(8, 8))
axs = axs.flatten()
model_types = [m for m in model_colours.keys() if m != 'HFFNN']
for ax, model_type in zip(axs, model_types):
    sns.stripplot(
        data=df[
            df['model_type'] == model_type
        ].sort_values('preprocessing.num_gesture_classes'),
        x='preprocessing.num_gesture_classes',
        y='val.pred_time_per_obs',
        hue='model_type',
#         dodge=True,
        s=2,
        legend=False,
        hue_order=list(model_colours.keys()),
        alpha=0.5,
        ax=ax,
    )
    ax.set_title(model_type)

# sns.stripplot(
#     data=df[
#         df['val.pred_time_per_obs'] < 0.02
#     ].sort_values('preprocessing.num_gesture_classes'),
#     hue='preprocessing.num_gesture_classes',
#     x='model_type',
#     y='val.pred_time_per_obs',
#     dodge=True,
#     s=2,
# #     hue_order=list(model_colours.keys()),
#     alpha=0.5,
#     ax=axs[1]
# )
plt.tight_layout()

## Example Confusion Matrices from baseline models

TODO: Also include models with perfect precision / perfect recall / perfect precision for non-g255 gestures / perfect precision for g255 / perfect recall for non-g255 gestures / perfect recall for g255

In [ ]:
# Load in the dataset/classifier
(
    X_trn, X_val, y_trn, y_val, dt_trn, dt_val
) = common.read_and_split_from_npz("../gesture_data/trn_20_10.npz")

### Plot a model that only predicts the non-gesture class

In [ ]:
# A function for plotting precision-recall + confusion matrices
def plt_pr_conf_mat(y_true, y_preds):
    """Given true and predicted labels, 
    create a confusion matrix and a precision-recall plot."""
    fig, axs = plt.subplots(1, 2, figsize=(10,5))
    if len(y_preds.shape) == 1:
        y_preds = np.array([y_preds])
    # Confusion matrix
    # Get the median confusion matrix
    n_classes = len(np.unique(y_true))
    cm_sum = np.zeros((n_classes, n_classes))
    
    for y_pred in y_preds:
        cm_sum += tf.math.confusion_matrix(y_true, y_pred).numpy()
    cm = cm_sum / len(y_preds)
    vis.conf_mat(cm, ax=axs[1])
    
    f1_sum = 0
    for y_pred in y_preds:
        f1_sum += sklearn.metrics.f1_score(y_true, y_pred, average='macro')
    f1 = f1_sum / len(y_preds)

    # Precision-recall plot
    recall_grid, precision_grid = np.meshgrid(
        np.linspace(0, 1, 100), 
        np.linspace(0, 1, 100)
    )
    f1_score = 2 * (precision_grid * recall_grid) / (precision_grid + recall_grid)

    contours = axs[0].contour(
        recall_grid, 
        precision_grid,
        f1_score, 
        levels=np.linspace(0.1, 1, 10), 
        colors='black',
        alpha=0.25
    )
    axs[0].clabel(contours, inline=True, fontsize=8, fmt='%.2f')

    ps = []
    rs = []
    for y_pred in y_preds:
        ps.append(sklearn.metrics.precision_score(y_true, y_pred, average='macro', zero_division=0))
        rs.append(sklearn.metrics.recall_score(y_true, y_pred, average='macro', zero_division=0))
    p = np.mean(ps)
    r = np.mean(rs)

    axs[0].scatter(
        ps, rs,
        color='black',
        s=5,
        alpha=0.5,
    )
    axs[0].set_xlabel('Precision')
    axs[0].set_ylabel('Recall')
    axs[0].set_xlim((-0.01, 1.01))
    axs[0].set_ylim((-0.01, 1.01))
    axs[0].plot([0,1], [0,1], color='black', alpha=.1)
#     axs[0].set_title(f'Precision-Recall Graph\n(contours indicate the $F_1$-score)')
    return fig, axs

#### Only predicts the non-gesture class

In [ ]:
# Only predicts 50
y_preds = np.random.randint(0, 51, size=(30, y_trn.shape[0]))
fig, axs = plt_pr_conf_mat(y_trn, y_preds);
plt.savefig('../../report/src/imgs/graphs/05_pr_conf_mat_only_50.pdf', bbox_inches='tight')

f1_scores = []
for y_pred in y_preds:
    f1_scores.append(sklearn.metrics.f1_score(y_trn, y_pred, average='macro', zero_division=0))
f1_scores = np.array(f1_scores)
print(f1_scores.min(), f1_scores.mean(), f1_scores.std(), f1_score.max())


#### Completely random predictions

In [ ]:
# Predicts randomly
y_preds = np.random.randint(0, 51, size=(30, y_trn.shape[0]))
fig, axs = plt_pr_conf_mat(y_trn, y_preds);
plt.savefig('../../report/src/imgs/graphs/05_pr_conf_mat_random_preds.pdf', bbox_inches='tight')

f1_scores = []
for y_pred in y_preds:
    f1_scores.append(sklearn.metrics.f1_score(y_trn, y_pred, average='macro', zero_division=0))
f1_scores = np.array(f1_scores)
print(f1_scores.min(), f1_scores.mean(), f1_scores.std(), f1_score.max())


#### Wrong orientation

In [ ]:
# Predicts wrong orientation
y_trn_repeated = np.repeat(y_trn[np.newaxis, :], 30, axis=0)
y_preds = np.where(
    # If the gesture is not g255
    y_trn_repeated != 50, 
    # Add/subtract some random multiple of 10
    y_trn_repeated + 10*np.random.randint(
        -(y_trn_repeated // 10), 
        +(5 - y_trn_repeated // 10), 
        y_trn_repeated.shape
    ), 
    # Else do nothing
    y_trn_repeated
)
fig, axs = plt_pr_conf_mat(y_trn, y_preds);
plt.savefig('../../report/src/imgs/graphs/05_pr_conf_mat_wrong_orientation.pdf', bbox_inches='tight')

f1_scores = []
for y_pred in y_preds:
    f1_scores.append(sklearn.metrics.f1_score(y_trn, y_pred, average='macro', zero_division=0))
f1_scores = np.array(f1_scores)
print(f1_scores.min(), f1_scores.mean(), f1_scores.std(), f1_score.max())


#### Wrong finger

In [ ]:
# Predicts wrong finger
y_trn_repeated = np.repeat(y_trn[np.newaxis, :], 30, axis=0)
y_preds = np.where(
    # If the gesture is not g255
    y_trn_repeated != 50, 
    # Then change only the last digit
    y_trn_repeated + np.random.randint(
        -np.mod(y_trn_repeated, 10), 
        +(10 - np.mod(y_trn_repeated, 10)),
        y_trn_repeated.shape
    ), 
    # else keep it the same
    y_trn_repeated
)

fig, axs = plt_pr_conf_mat(y_trn, y_preds);
plt.savefig('../../report/src/imgs/graphs/05_pr_conf_mat_wrong_finger.pdf', bbox_inches='tight')


f1_scores = []
for y_pred in y_preds:
    f1_scores.append(sklearn.metrics.f1_score(y_trn, y_pred, average='macro', zero_division=0))
f1_scores = np.array(f1_scores)
print(f1_scores.min(), f1_scores.mean(), f1_scores.std(), f1_score.max())


In [ ]:
# Predicts wrong finger (but correct hand)
y_trn_repeated = np.repeat(y_trn[np.newaxis, :], 30, axis=0)

y_preds = np.where(
    # If the gesture is not g255
    y_trn_repeated != 50, 
    # Then change only the last digit
    y_trn_repeated + np.random.randint(
        -np.mod(y_trn_repeated, 5), 
        +(5 - np.mod(y_trn_repeated, 5)),
        y_trn_repeated.shape
    ), 
    # else keep it the same
    y_trn_repeated
)

fig, axs = plt_pr_conf_mat(y_trn, y_preds);
plt.savefig('../../report/src/imgs/graphs/05_pr_conf_mat_wrong_finger_correct_hand.pdf', bbox_inches='tight')


f1_scores = []
for y_pred in y_preds:
    f1_scores.append(sklearn.metrics.f1_score(y_trn, y_pred, average='macro', zero_division=0))
f1_scores = np.array(f1_scores)
print(f1_scores.min(), f1_scores.mean(), f1_scores.std(), f1_score.max())


#### Predict 50 as a random gesture

In [ ]:
# Predicts 50 as random gesture
y_trn_repeated = np.repeat(y_trn[np.newaxis, :], 30, axis=0)
y_preds = np.where(
    # If the gesture IS g255
    y_trn_repeated == 50, 
    # Then choose a random non-g255 gesture
    np.random.randint(0, 50, y_trn_repeated.shape), 
    # else keep it the same
    y_trn_repeated
)
fig, axs = plt_pr_conf_mat(y_trn, y_preds);
plt.savefig('../../report/src/imgs/graphs/05_pr_conf_mat_no_gesture_50.pdf', bbox_inches='tight')


f1_scores = []
for y_pred in y_preds:
    f1_scores.append(sklearn.metrics.f1_score(y_trn, y_pred, average='macro', zero_division=0))
f1_scores = np.array(f1_scores)
print(f1_scores.min(), f1_scores.mean(), f1_scores.std(), f1_score.max())


#### High recall model

In [ ]:
# Predicts 50 as random gesture
y_trn_repeated = np.repeat(y_trn[np.newaxis, :], 30, axis=0)
y_preds = np.where(
    # If the gesture IS g255
    y_trn_repeated == 50, 
    # Then choose a random non-g255 gesture
    np.random.randint(0, 51, y_trn_repeated.shape), 
    # else keep it the same
    y_trn_repeated
)
fig, axs = plt_pr_conf_mat(y_trn, y_preds);
plt.savefig('../../report/src/imgs/graphs/05_pr_conf_mat_high_recall.pdf', bbox_inches='tight')

f1_scores = []
for y_pred in y_preds:
    f1_scores.append(sklearn.metrics.f1_score(y_trn, y_pred, average='macro', zero_division=0))
f1_scores = np.array(f1_scores)
print(f1_scores.min(), f1_scores.mean(), f1_scores.std(), f1_score.max())


#### High precision model

In [ ]:
# Predicts 50 as random gesture
y_trn_repeated = np.repeat(y_trn[np.newaxis, :], 5, axis=0)
y_preds = np.where(
    y_trn_repeated == 50,
    y_trn_repeated,
    50,
)

# np.clip(np.random.randint(-1, 1, size=y_trn_repeated.shape) + y_trn_repeated, 0, 50)
fig, axs = plt_pr_conf_mat(y_trn, y_preds);
# plt.savefig('../../report/src/imgs/graphs/05_pr_conf_mat_high_precision.pdf', bbox_inches='tight')

# f1_scores = []
# for y_pred in y_preds:
#     f1_scores.append(sklearn.metrics.f1_score(y_trn, y_pred, average='macro', zero_division=0))
# f1_scores = np.array(f1_scores)
# print(f1_scores.min(), f1_scores.mean(), f1_scores.std(), f1_score.max())


### Finally, plot all confusion matrices together

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(16, 10))
axs = axs.flatten()

# Random model:
y_pred = np.random.randint(0, 51, y_trn.shape)
cm_val = tf.math.confusion_matrix(y_trn, y_pred).numpy()
vis.conf_mat(cm_val, ax=axs[0])
f1 = sklearn.metrics.f1_score(y_trn, y_pred, average='macro')
axs[0].set_title(f"Random model\n$F_1$={f1:.4}")

# Only predicts 50
y_pred = np.full(y_trn.shape, 50)
cm_val = tf.math.confusion_matrix(y_trn, y_pred).numpy()
vis.conf_mat(cm_val, ax=axs[1])
f1 = sklearn.metrics.f1_score(y_trn, y_pred, average='macro')
axs[1].set_title(f"Only predicts 50\n$F_1$={f1:.4}")

# Perfect, but random orientation:
y_pred = np.where(
    # If the gesture is not g255
    y_trn != 50, 
    # Add/subtract some random multiple of 10
    y_trn + 10*np.random.randint(
        -(y_trn // 10), 
        +(5 - y_trn // 10), 
        y_trn.shape
    ), 
    # Else do nothing
    y_trn
)
cm_val = tf.math.confusion_matrix(y_trn, y_pred).numpy()
vis.conf_mat(cm_val, ax=axs[2])
f1 = sklearn.metrics.f1_score(y_trn, y_pred, average='macro')
axs[2].set_title(f"Perfect,\nbut random orientation\n$F_1$={f1:.4}")

# Perfect, but random finger:
y_pred = np.where(
    # If the gesture is not g255
    y_trn != 50, 
    # Then change only the last digit
    y_trn + np.random.randint(
        -np.mod(y_trn, 10), 
        +(10 - np.mod(y_trn, 10)),
        y_trn.shape
    ), 
    # else keep it the same
    y_trn
)
cm_val = tf.math.confusion_matrix(y_trn, y_pred).numpy()
vis.conf_mat(cm_val, ax=axs[3])
f1 = sklearn.metrics.f1_score(y_trn, y_pred, average='macro')
axs[3].set_title(f"Perfect,\nbut random finger\n$F_1$={f1:.4}")

# Perfect, but random finger (same hand):
y_pred = np.where(
    # If the gesture is not g255
    y_trn != 50, 
    # Then change only the last digit
    y_trn + np.random.randint(
        -np.mod(y_trn, 5), 
        +(5 - np.mod(y_trn, 5)),
        y_trn.shape
    ), 
    # else keep it the same
    y_trn
)
cm_val = tf.math.confusion_matrix(y_trn, y_pred).numpy()
vis.conf_mat(cm_val, ax=axs[4])
f1 = sklearn.metrics.f1_score(y_trn, y_pred, average='macro')
axs[4].set_title(f"Perfect,\nbut random finger on the correct hand\n$F_1$={f1:.4}")

# Never predicts 50
y_pred = np.where(
    # If the gesture IS g255
    y_trn == 50, 
    # Then choose a random non-g255 gesture
    np.random.randint(0, 50, y_trn.shape), 
    # else keep it the same
    y_trn
)
cm_val = tf.math.confusion_matrix(
    y_trn, 
    y_pred,
).numpy()
vis.conf_mat(cm_val, ax=axs[5])
f1 = sklearn.metrics.f1_score(y_trn, y_pred, average='macro')
axs[5].set_title(f"Perfect,\nbut predicts 50 as a random gesture\n$F_1$={f1:.4}")

plt.savefig('../../report/src/imgs/graphs/05_example_conf_mats.pdf')
plt.tight_layout()
plt.show()

## PCA Plots

### PCA decomposition only including the gesture classes

In [ ]:
(
    X_trn, X_val, y_trn, y_val, dt_trn, dt_val
) = common.read_and_split_from_npz("../gesture_data/trn_20_10.npz")


In [ ]:
%%script false --no-raise-error
from sklearn.manifold import TSNE
X_embedded = TSNE(
    n_components=2,
    learning_rate='auto',
    init='random',
    perplexity=3,
    verbose=True
).fit_transform(X_trn[y_trn != 50].reshape((X_trn[y_trn != 50].shape[0], 600)))

hues = np.array([ "0$^\circ$", "45$^\circ$", "90$^\circ$", "135$^\circ$", "180$^\circ$" ])
styles = np.array([ "L5", "L4", "L3", "L2", "L1", "R1", "R2", "R3", "R4", "R5" ])
fig, ax = plt.subplots(1, 1, figsize=(10,10), dpi=200)
argsort = np.argsort(y_trn[y_trn != 50])

sns.scatterplot(
    x=X_embedded[:, 0][argsort],
    y=X_embedded[:, 1][argsort],
    hue=hues[(y_trn[y_trn != 50][argsort] // 10)],
    style=styles[(y_trn[y_trn != 50][argsort] % 10)],
    s=10,
    alpha=0.5
#     ax=axs[0]
)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_reshaped = X_trn.reshape((X_trn.shape[0], 600))
X_tfrm = pca.fit_transform(X_reshaped[y_trn != 50])

argsort = np.argsort(y_trn[y_trn != 50])

hues = np.array([ "0$^\circ$", "45$^\circ$", "90$^\circ$", "135$^\circ$", "180$^\circ$" ])
styles = np.array([ "L5", "L4", "L3", "L2", "L1", "R1", "R2", "R3", "R4", "R5" ])

fig, axs = plt.subplots(1, 2, figsize=(16, 8), dpi=300)
sns.scatterplot(
    x=X_tfrm[:, 0][argsort],
    y=X_tfrm[:, 1][argsort],
    hue=hues[(y_trn[y_trn != 50][argsort] // 10)],
    style=styles[(y_trn[y_trn != 50][argsort] % 10)],
    s=10,
    ax=axs[0]
)
axs[0].set_title("PCA plot of the training data\nExcluding class 50")

pca = PCA(n_components=2)
X_reshaped = X_trn.reshape((X_trn.shape[0], 600))
X_tfrm = pca.fit_transform(X_reshaped)

colours = np.array([ "tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple"])

# fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=300)
mask = (y_trn == 50)
axs[1].scatter(
    X_tfrm[:, 0][mask],
    X_tfrm[:, 1][mask],
    color='black',
    alpha=0.1,
    s=5,
    edgecolor='none',
)
axs[1].scatter(
    X_tfrm[:, 0][~mask],
    X_tfrm[:, 1][~mask],
    color=colours[(y_trn[~mask] // 10)],
    alpha=0.5,
    s=5,
    edgecolor='none',
)
axs[1].set_title('PCA plot of the training data\n(class 50 in black)')

axs[0].set(
    xlabel='Principal Component 1',
    ylabel='Principal Component 2',
)
axs[1].set(
    xlabel='Principal Component 1',
    ylabel='Principal Component 2',
)
plt.tight_layout()

plt.savefig(
    '../../report/src/imgs/graphs/05_pca_plot.pdf',
    bbox_inches='tight'
)

### PCA plot showing just an interesting subset

In [ ]:
pca = PCA(n_components=2)
X_reshaped = X_trn.reshape((X_trn.shape[0], 600))
X_tfrm = pca.fit_transform(X_reshaped)

In [ ]:
colours = np.array([ "tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple"])

@interact(
    x_start=(-1500, 1500, 50),
    y_start=(-1500, 1500, 50),
    x_length=(-1500, 1500, 50),
    y_length=(-1500, 1500, 50),
)
def fn(x_start=-150, y_start=1000, x_length=500, y_length=500):
    x_finsh = x_start + x_length
    y_finsh = y_start + y_length

    fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=100)

    selection_mask = (
        (x_start <= X_tfrm[:, 0]) & (X_tfrm[:, 0] <= x_finsh) &
        (y_start <= X_tfrm[:, 1]) & (X_tfrm[:, 1] <= y_finsh)
    )
    X_subset = X_tfrm[selection_mask]
    y_subset = y_trn[selection_mask]
#     ax.scatter(
#         X_subset[:, 0],
#         X_subset[:, 1],
#         c='black',
#         alpha=0.1,
#     )
    
    y_mask = (y_trn == 50)
    ax.scatter(
        X_subset[:, 0][y_subset == 50],
        X_subset[:, 1][y_subset == 50],
        color='black',
        alpha=0.1,
        s=20,
        edgecolor='none',
    )
    ax.scatter(
        X_subset[:, 0][y_subset != 50],
        X_subset[:, 1][y_subset != 50],
        color=colours[(y_subset[y_subset != 50] // 10)],
        alpha=0.75,
#         s=5,
        edgecolor='none',
    )


### PCA plot that connects sequential datapoints

In [ ]:
pca = PCA(n_components=2)
X_reshaped = X_trn.reshape((X_trn.shape[0], 600))
X_tfrm = pca.fit_transform(X_reshaped)

order = np.argsort(dt_trn)
X_tfrm = X_tfrm[order]

In [ ]:


limit = 1000
@interact(start=(0, len(X_tfrm), 25), length=(0, len(X_tfrm), 50))
def fn(start=0, length=500):
    finsh = min(start + length, len(X_tfrm))
    fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=100)
    ax.plot(
        X_tfrm[:, 0][start:finsh],
        X_tfrm[:, 1][start:finsh],
        zorder=0,
        c='black',
        alpha=0.1,
    )
    sns.scatterplot(
        x=X_tfrm[:, 0][start:finsh],
        y=X_tfrm[:, 1][start:finsh],
        hue=dt_trn[order][start:finsh],
        legend=False,
        s=(10 + 90*(y_trn != 50)[order][start:finsh]),
        edgecolor=np.where((y_trn != 50)[order][start:finsh], 'black', 'none'),
        linewidth=.5,
    )
    
    idxs = np.nonzero(y_trn[order][start:finsh] != 50)[0]
    for idx in idxs:
        ax.text(
            X_tfrm[start:finsh][idx, 0],
            X_tfrm[start:finsh][idx, 1],
            y_trn[order][start:finsh][idx],
            va='center',
            ha='center',
        )
#     ax.set_xlim((
#         X_tfrm[:, 0].min() / 1.1,
#         X_tfrm[:, 0].max() * 1.1,
#     ))
#     ax.set_ylim((
#         X_tfrm[:, 1].min() / 1.1,
#         X_tfrm[:, 1].max() * 1.1,
#     ))

### PCA plot with ellipses

In [ ]:
pca = PCA(n_components=2)
X_reshaped = X_trn.reshape((X_trn.shape[0], 600))
X_tfrm = pca.fit_transform(X_reshaped[y_trn != 50])
y_tfrm = y_trn[y_trn != 50]

In [ ]:
?Ellipse

In [ ]:
# https://matplotlib.org/stable/gallery/statistics/confidence_ellipse.html
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1] / np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensional dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse(
        (0, 0), 
        width=ell_radius_x * 2, 
        height=ell_radius_y * 2,
        facecolor=facecolor, 
        **kwargs,
    )

    # Calculating the standard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the standard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)
    
#     print(f"{scale_x=}, {scale_y=}\n{mean_x=}, {mean_y=}")

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    
    return ax.add_patch(ellipse)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
# gidx = 14

hues = np.array([ "0$^\circ$", "45$^\circ$", "90$^\circ$", "135$^\circ$", "180$^\circ$" ])
styles = np.array([ "L5", "L4", "L3", "L2", "L1", "R1", "R2", "R3", "R4", "R5" ])
markers = ['o', 's', 'D', '^', 'v', '>', '<', 'p', 'H', '+']
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple']

for gidx in range(50):
    ax.scatter(
        X_tfrm[y_tfrm == gidx, 0],
        X_tfrm[y_tfrm == gidx, 1],
        color=colors[gidx // 10],
        marker=markers[gidx % 10],
        s=5,
        alpha=0.5,
        label=gidx
    )
    confidence_ellipse(
        X_tfrm[y_tfrm == gidx, 0],
        X_tfrm[y_tfrm == gidx, 1],
        ax,
        n_std=2,
        edgecolor=colors[gidx // 10],
        alpha=0.5,
    )
plt.legend()

## Visualise mis-predictions

1. Load in a continuous dataset
2. Load in a classifier
3. Use the classifier to make predictions on the dataset
4. Visualise the mispredictions, but *with context*

### Load in a model for which to evaluate the mis-predictions

In [ ]:
# Load in the dataset/classifier
(
    X_trn, X_val, y_trn, y_val, dt_trn, dt_val
) = common.read_and_split_from_npz("../gesture_data/trn_20_10.npz")

clf = models.load_tf('../src/saved_models/ffnn_2023-09-18T14:05:16.363404')
const = common.read_constants('../src/constants.yaml')
sensor_names = list(const['sensors'].values())

### Visualise True and Mispredicted gestures

Plot all the observations which have the ground truth being gesture 255 but the model is not predicting g255

In [ ]:
y_pred = clf.predict(X_val)

for gidx in np.unique(y_val):
    if gidx == 50: continue
    pred_indxs = np.nonzero((y_val == 50) & (y_pred == gidx))[0]
    true_indxs = np.nonzero(y_val == gidx)[0]
    axs = vis.cmp_ts(
        X_val[true_indxs],
    )
    vis.cmp_ts(
        X_val[pred_indxs],
        color='tab:red',
        axs=axs,
    )

#     distances = np.abs(true_indxs[:, np.newaxis] - pred_indxs).min(axis=0)

    plt.suptitle(f'Model predicted {gidx}, ground truth: 50 \
                 \nGesture {gidx} in grey, mispredicted in red ({len(pred_indxs)} observations) \
                 \nindices: {pred_indxs}')
    plt.tight_layout()
#     plt.savefig(f'../src/notebooks/pred_{gidx:0>2}_truth_50.pdf', bbox_inches='tight')
    plt.show()
#     if gidx > 5:
    break


# Interactive plot to see data at a certain time

In [ ]:

df = read.read_data(
    '../gesture_data/train/', 
#     constants_path='../src/constants.yaml',
)
df['gidx'] = df['gesture'].apply(lambda g: int(g[-4:]) if g != 'gesture0255' else 50)

@interact(dt='2022-10-08T20:23:46.665276000')
def fn(dt='2022-10-08T20:23:46.665276000'):
    try:
        dt = pd.to_datetime(dt)
    except Exception as e:
        print(e)
        return
#     fig, ax = plt.subplots(1, 1, figsize=(20, 6))
    mask = df['datetime'].between(
        dt - pd.to_timedelta(1, 'second'),
        dt + pd.to_timedelta(1, 'second')
    )
    
    vis.cmp_ts(
        [df.loc[mask, sensor_names].values]
    )
#     ax.plot(
#         df.loc[mask, sensor_names].values
#     )
#     dt_labels = df.loc[mask, 'datetime']
#     gidx_labels = df.loc[mask, 'gidx']
#     ax.set_xticks(range(len(dt_labels)))
#     ax.set_xticklabels([
#         f'{gidx_label} {str(dt_label)[5:-3]}'
#         for dt_label, gidx_label
#         in zip(dt_labels, gidx_labels)
#     ], rotation=90)
# 2022-10-08T20:23:46.665276000

# Plot a CSV file + predictions

In [ ]:

ellipses = calculate_prediction_ellipse(
    X_tfrm[y_tfrm == 0, 0],
    X_tfrm[y_tfrm == 0, 1],
)
ellipses

In [ ]:
csv_path = '../gesture_data/saved_from_cli_alphabet.csv'
model_dir = '../src/saved_models/ffnn_2023-10-04T12:02:09.749144'

sensors = list(common.read_constants('./constants.yaml')["sensors"].values())
df = pd.read_csv(
    csv_path,
    names=["datetime", "gesture"] + sensors,
    parse_dates=["datetime"],
    date_format='ISO8601',
)
df['file'] = csv_path
X, y, dt = common.make_windows(
    df,
    20,
    constants_path='../src/constants.yaml',
    pbar=tqdm.tqdm(total=len(df), desc="Making windows"),
)
clf = models.load_tf(model_dir)
y_pred = clf.predict(X)
y_pred_probs = clf.predict_proba(X)

In [ ]:
# clf=None
@interact(start=(0, len(df), 20), duration=(0, len(df), 20))
def fn(start=0, duration=4000):
    
    fig, axs = plt.subplots(3 if clf is not None else 2, 1, figsize=(10, 5))
    for i in range(X.shape[-1]):
        axs[0].plot(
            X[start:start+duration, 0, i],
            alpha=0.5,
            c=['tab:red', 'tab:green', 'tab:blue'][i%3],
            lw=1,
        )
    sns.heatmap(
        X[start:start+duration, 0, :].T,
        cmap='jet',
        ax=axs[1],
        cbar=False,
        vmin=290,
        vmax=910,
    )

    if clf is not None:
        sns.heatmap(
            y_pred_probs[start-10:start+duration-10, :].T,
            vmin=0,
            vmax=1,
            ax=axs[2],
            cbar=False,
        )
        axs[2].set(
            ylabel='Predicted\ngesture',
        )

    plt.subplots_adjust(hspace=0.05)
    
    axs[0].set(
        ylabel='Sensor value',
        xticks=[],
        ylim=(250, 950)
    )
    axs[1].set(
        ylabel='Sensor number',
        xticks=[],
    )
    axs[0].margins(0)
    with_clf =' and model predictions' if clf is not None else ''
    axs[0].set_title(f'Sensor values{with_clf} over time at {np.round(start/40, 2)}s\n(duration: {np.round(duration/40, 2)} seconds')
#     plt.savefig(
#         f'sensors_over_time_{start}_{duration}.pdf',
#         bbox_inches='tight',
#     )
    

# Misc Research Chapter Plots

### Read in all the data

In [ ]:
df = read.read_data(
    '../gesture_data/train/', 
    constants_path='../src/constants.yaml'
)
df['gidx'] = df['gesture'].apply(lambda g: int(g[-4:]) if g != 'gesture0255' else 50)
const = common.read_constants('../src/constants.yaml')
sensor_names = list(const['sensors'].values())

## Correlations between the different gestures

In [ ]:
timestep = 0
# Load in the dataset/classifier
(
    X_trn, X_val, y_trn, y_val, dt_trn, dt_val
) = common.read_and_split_from_npz("../gesture_data/trn_20_10.npz")
const = common.read_constants('../src/constants.yaml')
sensor_names = list(const['sensors'].values())
X_data = X_trn[y_trn != 50][:, timestep, :]
y_data = y_trn[y_trn != 50]

In [ ]:
fig, axs = plt.subplots(
    5, 10,
    figsize=(40, 20),
    dpi=200,
)
for i in range(5):
    print(f'gesture {i}_', flush=True)
    for j in range(10):
        sns.heatmap(
            pd.DataFrame(X_data[y_data == (i * 10 + j)]).corr(),
            vmin=-1, 
            vmax=1,
            center=0,
            cbar=False,
            ax=axs[i, j],
            xticklabels=[s.upper() for s in sensor_names],
            yticklabels=[s.upper() for s in sensor_names],
            square=True,
        )
        axs[i, j].set_title(f'Gesture {i * 10 + j}')
plt.subplots_adjust(hspace=0.35, wspace=0.25)

In [ ]:
sns.heatmap(
    pd.DataFrame(X_data).corr(),
    vmin=-1, 
    vmax=1,
    center=0,
#     cbar=False,
    xticklabels=[s.upper() for s in sensor_names],
    yticklabels=[s.upper() for s in sensor_names],
    square=True,
)
plt.xlabel('Sensor')
plt.ylabel('Sensor')
plt.title('Correlations between sensors\n(over all training data)')
plt.savefig(
    '../../report/src/imgs/graphs/05_correlations.pdf',
    bbox_inches='tight'
)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(12, 4))
axis = ['X', 'Y', 'Z']
for i in range(3):
    sns.heatmap(
        pd.DataFrame(X_data[:, i::3]).corr(),
        vmin=-1, 
        vmax=1,
        center=0,
        cbar=False,
        square=True,
        ax=axs[i]
    )
    axs[i].set_title(f'Correlations between {axis[i]}-axis sensors\n(over all training data)')
plt.tight_layout()
plt.show()


## Time-series heatmap + line plots

In [ ]:

def plt_subset(s, f):
    df = read.read_data(
        '../gesture_data/train/', 
        constants_path='../src/constants.yaml'
    )
    df['gidx'] = df['gesture'].apply(lambda g: int(g[-4:]) if g != 'gesture0255' else 50)
    const = common.read_constants('../src/constants.yaml')
    sensor_names = list(const['sensors'].values())
    data = df[sensor_names].values[s:f]
    
    fig, axs = plt.subplots(2, 1, figsize=(10, 6))
    sns.heatmap(
        data.T,
        ax=axs[0],
        cbar=False
    )

    axs[1].plot(
        data,
        color='black',
        alpha=0.2,
        linewidth=1,
    )
    plt.margins(0)
    plt.show()
# plt_subset(91_000, 95_000)
plt_subset(93_000, 93_200)

## Histogram of class distributions

In [ ]:
df['gidx'].hist()
plt.show()
df.loc[df['gidx'] != 50, 'gidx'].hist()
df['gidx'].value_counts() / len(df['gidx']) * 100

## All observations of one gesture

In [ ]:
gidx = 0
before = 10
after = 10
idxs = np.nonzero(df['gidx'] == gidx)[0][:, np.newaxis] + np.arange(-before, after+1)

vis.cmp_ts(df[sensor_names].values[idxs + 10]);
plt.tight_layout()
plt.show()

In [ ]:
np.arange(-before, after+1)

## 3D plot of the raw acceleration data

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
vals = df[['l5x', 'l5y', 'l5z']].values[:10000]
ax.plot(
    vals[:, 0], 
    vals[:, 1], 
    vals[:, 2], 
    label='3D Line'
)